##### Grading Feedback

# IST 718: Big Data Analytics

- Professor: Willard Williamson <wewillia@syr.edu>
- Faculty Assistant: Vidushi Mishra <vmishr01@syr.edu>
- Faculty Assistant: Pranav Kottoli Radhakrishna <pkottoli@syr.edu>
## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any part of your answers from your classmates.  Short code snippets are allowed from the internet.  Code from the class text books or class provided code can be copied in its entirety.__
- Do not modify cells marked as grading cells or marked as do not modify.
- Before submitting your work, remember to check for run time errors with the following procedure:
`Runtime `$\rightarrow$ Factory reset runtime followed by Runtime $\rightarrow$ Run All.  All runtime errors will result in a minimum penalty of half off.
- Google Colab is the official class runtime environment so you should test your code on Colab before submission.
- All plots shall include descriptive title and axis labels.  Plot legends shall be included where possible.  Unless stated otherwise, plots can be made using any Python plotting package.  It is understood that spark data structures must be converted to something like numpy or pandas prior to making plots.  All required mathematical operations, filtering, selection, etc., required by a homework question shall be performed in spark prior to converting to numpy or pandas.
- Grading feedback cells are there for graders to provide feedback to students.  Don't change or remove grading feedback cells.
- Don't add or remove files from your git repo.
- Do not change file names in your repo.  This also means don't change the title of the ipython notebook.
- You are free to add additional code cells around the cells marked `your code here`.
- We reserve the right to take points off for operations that are extremely inefficient or "heavy weight".  This is a big data class and extremely inefficient operations make a big difference when scaling up to large data sets.  For example, the spark dataframe collect() method is a very heavy weight operation and should not be used unless it there is a real need for it.  An example where collect() might be needed is to get ready to make a plot after filtering a spark dataframe.
- import * is not allowed because it is considered a very bad coding practice and in some cases can result in a significant delay (which slows down the grading process) in loading imports.  For example, the statement `from sympy import *` is not allowed.  You must import the specific packages that you need. 
- The graders reserve the right to deduct points for subjective things we see with your code.  For example, if we ask you to create a pandas data frame to display values from an investigation and you hard code the values, we will take points off for that.  This is only one of many different things we could find in reviewing your code.  In general, write your code like you are submitting it for a code peer review in industry.  
- Level of effort is part of our subjective grading.  For example, in cases where we ask for a more open ended investigation, some students put in significant effort and some students do the minimum possible to meet requirements.  In these cases, we may take points off for students who did not put in much effort as compared to students who put in a lot of effort.  We feel that the students who did a better job deserve a better grade.  We reserve the right to invoke level of effort grading at any time.
- Only use spark, spark machine learning, spark data frames, RDD's, and map reduce to solve all problems unless instructed otherwise.
- Your notebook must run from start to finish without requiring manual input by the graders.  For example, do not mount your personal Google drive in your notebook as this will require graders to perform manual steps.  In short, your notebook should run from start to finish with no runtime errors and no need for graders to perform any manual steps.


## Note that this notebook is expected to run in the Google Colab environment.  All grading for this assignment will take place exclusively in Google Colab.

This homework proves that diamonds are forever.  In homework 3, we used linear regression to predict diamond prices and evaluated model performance using MSE as the scoring metric.  In this homework, we are going to use the same diamonds data set but this time use decision trees and deep learning to see if we can improve upon the linear regression performance from homework 3.

# Diamonds Data
Just to prove that diamonds are forever, we are going to revisit the diamonds data set.  This homework assignment will use the diamonds dataset to explore random forest decision tree models.

The diamonds.csv data set contains 10 columns:
- carat: Carat weight of the diamond
- cut: Describes cut quality of the diamond. Quality in increasing order Fair, Good, Very Good, Premium, Ideal
- color: Color of the diamond, with D being the best and J the worst
- clarity: How obvious inclusions are within the diamond:(in order from best to worst, FL = flawless, I3= level 3 inclusions) FL,IF, VVS1, etc.  See this web site for an exhaustive ranking of [clarity](https://4cs.gia.edu/en-us/diamond-clarity/?gclid=Cj0KCQjwnqH7BRDdARIsACTSAduMoc2KQbXkO94BxCfBNC5X8YyjAYcFpWThKQMW46cQj_3p0pZ0o84aAuagEALw_wcB).  The web site has a nice sliding scale you can drag to see the relationship between clarity grades.
- depth: depth % - The height of a diamond, measured from the culet to the table, divided by its average girdle diameter
- table: table% -  The width of the diamond's table expressed as a percentage of its average diameter
- price: The price of the diamond
- x: Length (mm)
- y: Width (mm)
- z: Height (mm)

In [75]:
# Grading Cell
enable_grid_search = False

The following cell is used to read the diamonds data set into the colab environment.  Do not change or modify the following cell.

In [2]:
%%bash
# Do not change or modify this file
# Need to install pyspark
# if pyspark is already installed, will print a message indicating pyspark already isntalled
pip install pyspark

# Download the data files from github
# If the data file does not exist in the colab environment
if [[ ! -f ./quotes_by_char.csv ]]; then 
   # download the data file from github and save it in this colab environment instance
   wget https://raw.githubusercontent.com/wewilli1/ist718_data/master/diamonds.csv  
fi

  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=94a0c212d7feb19b907baf01db0dffeafa31d6aee15f6c08c07ecbe5dda1bcd3
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


--2020-11-23 02:42:27--  https://raw.githubusercontent.com/wewilli1/ist718_data/master/diamonds.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3192560 (3.0M) [text/plain]
Saving to: ‘diamonds.csv’

     0K .......... .......... .......... .......... ..........  1% 3.96M 1s
    50K .......... .......... .......... .......... ..........  3% 9.63M 1s
   100K .......... .......... .......... .......... ..........  4% 3.90M 1s
   150K .......... .......... .......... .......... ..........  6% 5.90M 1s
   200K .......... .......... .......... .......... ..........  8% 30.6M 0s
   250K .......... .......... .......... .......... ..........  9% 17.4M 0s
   300K .......... .......... .......... .......... .......... 11% 35.5M 0s
   350K .......... .......... ........

# Question 0 (0 pts)
Please provide the following the data so we can easily correlate your notebook with the grade book:
- Your Name: Ruiwei Zhang
- Your github user name: ValanZRW
- Your SU email address: rzhang72@syr.edu

Your grade for grid search problems in this assignment will be determined in part on level of effort and your model performance results as compared to other students in the class.

# Question 1 (10 pts)
Read the diamonds.csv file into a spark data frame named `diamonds_df`.  Perform feature engineering as needed for training decision trees.  Name the new data frame diamonds_df_xformed.

In [3]:
# your code here
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import lower, col
spark = SparkSession \
  .builder \
  .master("local[*]")\
  .config("spark.memory.fraction", 0.8) \
  .config("spark.executor.memory", "12g") \
  .config("spark.driver.memory", "12g")\
  .config("spark.memory.offHeap.enabled",'true')\
  .config("spark.memory.offHeap.size","12g")\
  .getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

diamonds_df = spark.read.csv('diamonds.csv', header=True, inferSchema=True)

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexerModel
from pyspark.ml.feature import VectorIndexer

feature_engineering_pipe = Pipeline(stages = [StringIndexerModel.from_labels(["Fair", "Good", "Very Good", "Premium", "Ideal"], inputCol="cut", outputCol="cut_idx"), 
                         StringIndexerModel.from_labels(["D", "E", "F", "G", "H", "I", "J"], inputCol="color", outputCol="color_idx"),
                         StringIndexerModel.from_labels(["Fl", "IF", "VVS1", "VVS2", "VS1", "VS2", "SI1", "SI2", "I1", "I2", "I3"], inputCol="clarity", \
                                                                            outputCol="clarity_idx"),])

diamonds_df_xformed = feature_engineering_pipe.fit(diamonds_df).transform(diamonds_df)\
                      .drop('_c0', 'cut', 'color', 'clarity')\
                      .withColumnRenamed('cut_idx', 'cut')\
                      .withColumnRenamed('color_idx', 'color')\
                      .withColumnRenamed('clarity_idx', 'clarity')


In [4]:
# Grading Cell - do not modify
display(diamonds_df_xformed.toPandas().head())

,carat,depth,table,price,x,y,z,cut,color,clarity
0,0.23,61.5,55.0,326,3.95,3.98,2.43,4.0,1.0,7.0
1,0.21,59.8,61.0,326,3.89,3.84,2.31,3.0,1.0,6.0
2,0.23,56.9,65.0,327,4.05,4.07,2.31,1.0,1.0,4.0
3,0.29,62.4,58.0,334,4.20,4.23,2.63,3.0,5.0,5.0
4,0.31,63.3,58.0,335,4.34,4.35,2.75,1.0,6.0,7.0


##### Grading Feedback Cell

The following questions will create a random forest regressor model, train the model using a grid search, and use the model for inference.  The goal is to see if we can improve upon the linear regression score from homework 3. You can find the spark documentation for the random forest regressor [here](https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-regression).

# Question 2 (20 pts)
Create and train your random forest regressor model using a grid search in the cell below.  You are free to use K-Fold Cross validation if you wish.  Your grid search must be entirely encapsulated in the `if enable_grid_search` if statement.  The `enable_grid_search` Boolean is defined in a grading cell above.  You will disable the grid search before you submit by setting enable_grid_search to false.  Setting enable_grid_search to false should not result in a runtime error.  You will not receive full credit if any part of your grid search is outside of the if statement or if runtime errros result from setting the `enable_grid_search` variable to false.

In [78]:
# your code here
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import functions as F

training_df, validation_df, testing_df = diamonds_df_xformed.randomSplit([0.6, 0.3, 0.1])
va = VectorAssembler(inputCols=list(set(diamonds_df_xformed.columns)-set(['price'])), outputCol='features')

evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='price', metricName='mse')

if enable_grid_search:
  grid = ParamGridBuilder().\
      addGrid(rf.maxDepth, [5, 10]).\
      addGrid(rf.numTrees, [50, 75]).\
      build()
  
  all_models = []
  for j in range(len(grid)):
    print("Fitting model {}".format(j+1))
    model = rf_pipeline.fit(training_df, grid[j])
    all_models.append(model)

  import numpy as np
  mse = [evaluator.evaluate(m.transform(validation_df)) for m in all_models]

  best_model_idx = np.argmin(mse)
  best_model = all_models[best_model_idx]

  print("Best accuracy: ", mse[best_model_idx])
  print("Best grid: ", grid[best_model_idx])

  pass

##### Grading Feedback Cell

# Question 3 (20 pts)
Create a pipeline named `best_pipe` that hard codes the tuning parameters from the best model found by the grid search in question 2 above.  Train and test best_pipe.  Do not use k-fold cross validation in question 3.  Clearly print the resulting train and test MSE for best_pipe so it's easy for the graders to see your resulting MSEs.

In [79]:
# Your code here

best_rf = RandomForestRegressor(featuresCol="features",labelCol='price',maxDepth=10,numTrees=75)
best_pipe = Pipeline(stages=[va, best_rf]).fit(training_df)

print("MSE: ", evaluator.evaluate(best_pipe.transform(testing_df)))


MSE:  509707.09139480535


##### Grading Feedback Cell

# Question 4 (20 pts)
Use your best_pipe pipeline in question 3 for inference.  Create a pandas data frame named `rf_feature_importance` which contains 2 columns: `feature`, and `importance`.  Load the feature column with the feature name and the importance column with the feature importance score as determined by the random forest model. Sort the feature importances from high to low such that the most important feature is in the first row of the data frame.

In [72]:
# your code here
import pandas as pd

feature = best_pipe.stages[0].getInputCols()
importance = best_pipe.stages[-1].featureImportances.toArray()

rf_feature_importance = pd.DataFrame({'feature':best_pipe.stages[0].getInputCols(), 'importance':best_pipe.stages[-1].featureImportances.toArray()})
rf_feature_importance = rf_feature_importance.sort_values(by = 'importance', ascending=False)

In [73]:
# grading cell - do not modify
display(rf_feature_importance)

,feature,importance
3,y,0.286600
4,carat,0.285132
1,x,0.236151
8,z,0.102912
2,clarity,0.053135
5,color,0.026447
7,table,0.003600
0,cut,0.003147
6,depth,0.002875


##### Grading Feedback Cell

# Question 5 (20 pts)
Write code to print the decision logic for any of the trees in the forest from the best_pipe pipeline.  Copy the printed decision text to the tree printout markdown cell below and retain the same formatting and indentation as the code printout so it's easy for the graders to view the data.  You need to double click the "Your Decision Tree Print Out Here" markdown cell and paste your output inside the two sets of triple quotes. The triple quotes are jupyter markdown indicating you want to present code.  Essentially, replace the text inside the triple quotes with your tree printout.  Solutions that do not maintain readable formatting will not receive full credit.

Add comments to the markdown cell below describing how the root node is split:  Describe 2 things in the markdown cell.  1) What specific predictor variable is being split and what is the value that determines the left / right split.  2) We need you to paste the tree decision logic output from your run in the markdown cell because the top level split may change from run to run.  If the graders run your notebook, the top level split for the tree may be different than the top level split from when you made the run.  Describe why the top level predictor changes from run to run.


In [74]:
# your code here
print(best_pipe.stages[-1].trees[0].toDebugString)

DecisionTreeRegressionModel: uid=dtr_a9c2bceb6d22, depth=10, numNodes=1601, numFeatures=9
  If (feature 4 <= 0.995)
   If (feature 3 <= 5.525)
    If (feature 8 <= 3.135)
     If (feature 1 <= 4.6850000000000005)
      If (feature 5 in {4.0,5.0,6.0})
       If (feature 2 in {4.0,5.0,6.0,7.0})
        If (feature 8 <= 2.7350000000000003)
         If (feature 0 in {0.0,1.0,2.0,4.0})
          If (feature 4 <= 0.305)
           If (feature 4 <= 0.28500000000000003)
            Predict: 442.7931034482759
           Else (feature 4 > 0.28500000000000003)
            Predict: 481.27669902912623
          Else (feature 4 > 0.305)
           If (feature 5 in {5.0,6.0})
            Predict: 473.5632183908046
           Else (feature 5 not in {5.0,6.0})
            Predict: 540.5526315789474
         Else (feature 0 not in {0.0,1.0,2.0,4.0})
          If (feature 1 <= 4.385)
           If (feature 1 <= 4.205)
            Predict: 418.3333333333333
           Else (feature 1 > 4.205)
            

 ```
DecisionTreeRegressionModel: uid=dtr_a9c2bceb6d22, depth=10, numNodes=1601, numFeatures=9
  If (feature 4 <= 0.995)
   If (feature 3 <= 5.525)
    If (feature 8 <= 3.135)
     If (feature 1 <= 4.6850000000000005)
      If (feature 5 in {4.0,5.0,6.0})
       If (feature 2 in {4.0,5.0,6.0,7.0})
        If (feature 8 <= 2.7350000000000003)
         If (feature 0 in {0.0,1.0,2.0,4.0})
          If (feature 4 <= 0.305)
           If (feature 4 <= 0.28500000000000003)
            Predict: 442.7931034482759
           Else (feature 4 > 0.28500000000000003)
            Predict: 481.27669902912623
          Else (feature 4 > 0.305)
           If (feature 5 in {5.0,6.0})
            Predict: 473.5632183908046
           Else (feature 5 not in {5.0,6.0})
            Predict: 540.5526315789474
         Else (feature 0 not in {0.0,1.0,2.0,4.0})
          If (feature 1 <= 4.385)
           If (feature 1 <= 4.205)
            Predict: 418.3333333333333
           Else (feature 1 > 4.205)
            Predict: 547.0944881889764
          Else (feature 1 > 4.385)
           If (feature 5 in {5.0,6.0})
            Predict: 564.1395348837209
           Else (feature 5 not in {5.0,6.0})
            Predict: 614.2881355932203
        Else (feature 8 > 2.7350000000000003)
         If (feature 4 <= 0.355)
          If (feature 0 in {0.0,1.0,2.0,4.0})
           If (feature 2 in {6.0,7.0})
            Predict: 495.19718309859155
           Else (feature 2 not in {6.0,7.0})
            Predict: 580.5438596491229
          Else (feature 0 not in {0.0,1.0,2.0,4.0})
           If (feature 4 <= 0.325)
            Predict: 570.6363636363636
           Else (feature 4 > 0.325)
            Predict: 645.54
         Else (feature 4 > 0.355)
          If (feature 7 <= 57.05)
           If (feature 8 <= 2.895)
            Predict: 603.8305084745763
           Else (feature 8 > 2.895)
            Predict: 650.7073170731708
          Else (feature 7 > 57.05)
           If (feature 8 <= 2.895)
            Predict: 676.1206896551724
           Else (feature 8 > 2.895)
            Predict: 724.7333333333333
       Else (feature 2 not in {4.0,5.0,6.0,7.0})
        If (feature 3 <= 4.215)
         If (feature 0 in {1.0,2.0})
          If (feature 8 <= 2.585)
           If (feature 7 <= 56.05)
            Predict: 424.8888888888889
           Else (feature 7 > 56.05)
            Predict: 485.3333333333333
          Else (feature 8 > 2.585)
           Predict: 504.0
         Else (feature 0 not in {1.0,2.0})
          If (feature 8 <= 2.585)
           If (feature 5 in {5.0,6.0})
            Predict: 454.22222222222223
           Else (feature 5 not in {5.0,6.0})
            Predict: 528.8461538461538
          Else (feature 8 > 2.585)
           Predict: 586.0
        Else (feature 3 > 4.215)
         If (feature 1 <= 4.385)
          If (feature 1 <= 4.295)
           If (feature 4 <= 0.28500000000000003)
            Predict: 472.125
           Else (feature 4 > 0.28500000000000003)
            Predict: 617.2073170731708
          Else (feature 1 > 4.295)
           If (feature 5 in {5.0,6.0})
            Predict: 616.3018867924528
           Else (feature 5 not in {5.0,6.0})
            Predict: 732.5606936416185
         Else (feature 1 > 4.385)
          If (feature 0 in {1.0,2.0,4.0})
           If (feature 1 <= 4.585)
            Predict: 740.2228571428572
           Else (feature 1 > 4.585)
            Predict: 848.7205882352941
          Else (feature 0 not in {1.0,2.0,4.0})
           If (feature 2 in {2.0,3.0})
            Predict: 830.6
           Else (feature 2 not in {2.0,3.0})
            Predict: 939.4166666666666
      Else (feature 5 not in {4.0,5.0,6.0})
       If (feature 3 <= 4.215)
        If (feature 0 in {2.0})
         If (feature 2 in {4.0,5.0})
          If (feature 6 <= 63.05)
           If (feature 7 <= 60.05)
            Predict: 438.35526315789474
           Else (feature 7 > 60.05)
            Predict: 377.75
          Else (feature 6 > 63.05)
           If (feature 5 in {0.0,1.0,2.0})
            Predict: 437.9
           Else (feature 5 not in {0.0,1.0,2.0})
            Predict: 671.0
         Else (feature 2 not in {4.0,5.0})
          If (feature 5 in {0.0,1.0,3.0})
           If (feature 7 <= 59.1)
            Predict: 534.20625
           Else (feature 7 > 59.1)
            Predict: 500.80645161290323
          Else (feature 5 not in {0.0,1.0,3.0})
           If (feature 1 <= 4.205)
            Predict: 567.6388888888889
           Else (feature 1 > 4.205)
            Predict: 641.0
        Else (feature 0 not in {2.0})
         If (feature 5 in {3.0})
          If (feature 4 <= 0.28500000000000003)
           If (feature 7 <= 55.05)
            Predict: 476.44444444444446
           Else (feature 7 > 55.05)
            Predict: 530.3870967741935
          Else (feature 4 > 0.28500000000000003)
           If (feature 6 <= 62.05)
            Predict: 669.0
           Else (feature 6 > 62.05)
            Predict: 749.3333333333334
         Else (feature 5 not in {3.0})
          If (feature 2 in {4.0,5.0,6.0})
           If (feature 4 <= 0.28500000000000003)
            Predict: 482.13392857142856
           Else (feature 4 > 0.28500000000000003)
            Predict: 638.375
          Else (feature 2 not in {4.0,5.0,6.0})
           If (feature 7 <= 61.1)
            Predict: 691.9940476190476
           Else (feature 7 > 61.1)
            Predict: 501.6666666666667
       Else (feature 3 > 4.215)
        If (feature 2 in {4.0,5.0,6.0,7.0,8.0})
         If (feature 4 <= 0.325)
          If (feature 4 <= 0.305)
           If (feature 2 in {6.0,7.0,8.0})
            Predict: 531.6843971631206
           Else (feature 2 not in {6.0,7.0,8.0})
            Predict: 688.0967741935484
          Else (feature 4 > 0.305)
           If (feature 2 in {6.0,7.0,8.0})
            Predict: 588.9893899204244
           Else (feature 2 not in {6.0,7.0,8.0})
            Predict: 735.60125
         Else (feature 4 > 0.325)
          If (feature 1 <= 4.585)
           If (feature 2 in {6.0,7.0,8.0})
            Predict: 625.6994047619048
           Else (feature 2 not in {6.0,7.0,8.0})
            Predict: 798.295415959253
          Else (feature 1 > 4.585)
           If (feature 0 in {1.0,2.0})
            Predict: 779.219512195122
           Else (feature 0 not in {1.0,2.0})
            Predict: 856.860824742268
        Else (feature 2 not in {4.0,5.0,6.0,7.0,8.0})
         If (feature 1 <= 4.425)
          If (feature 4 <= 0.28500000000000003)
           If (feature 0 in {2.0})
            Predict: 611.4318181818181
           Else (feature 0 not in {2.0})
            Predict: 712.4583333333334
          Else (feature 4 > 0.28500000000000003)
           If (feature 1 <= 4.295)
            Predict: 796.5340314136125
           Else (feature 1 > 4.295)
            Predict: 887.3124324324324
         Else (feature 1 > 4.425)
          If (feature 3 <= 4.595)
           If (feature 2 in {3.0})
            Predict: 900.9859154929577
           Else (feature 2 not in {3.0})
            Predict: 1023.03125
          Else (feature 3 > 4.595)
           If (feature 6 <= 60.75)
            Predict: 992.1333333333333
           Else (feature 6 > 60.75)
            Predict: 1095.3232323232323
     Else (feature 1 > 4.6850000000000005)
      If (feature 4 <= 0.495)
       If (feature 1 <= 4.845)
        If (feature 5 in {5.0,6.0})
         If (feature 7 <= 55.95)
          If (feature 0 in {4.0})
           If (feature 2 in {3.0,4.0,5.0,6.0,7.0})
            Predict: 677.7258064516129
           Else (feature 2 not in {3.0,4.0,5.0,6.0,7.0})
            Predict: 893.2222222222222
          Else (feature 0 not in {4.0})
           If (feature 2 in {3.0,6.0})
            Predict: 865.6666666666666
           Else (feature 2 not in {3.0,6.0})
            Predict: 1041.0
         Else (feature 7 > 55.95)
          If (feature 6 <= 61.45)
           If (feature 1 <= 4.765)
            Predict: 697.0
           Else (feature 1 > 4.765)
            Predict: 823.0
          Else (feature 6 > 61.45)
           If (feature 2 in {4.0,5.0,6.0,7.0})
            Predict: 738.0212765957447
           Else (feature 2 not in {4.0,5.0,6.0,7.0})
            Predict: 963.9090909090909
        Else (feature 5 not in {5.0,6.0})
         If (feature 6 <= 62.849999999999994)
          If (feature 0 in {0.0,1.0,2.0,3.0})
           If (feature 2 in {5.0,6.0,7.0})
            Predict: 849.0745192307693
           Else (feature 2 not in {5.0,6.0,7.0})
            Predict: 1088.257225433526
          Else (feature 0 not in {0.0,1.0,2.0,3.0})
           If (feature 2 in {4.0,5.0,6.0,7.0})
            Predict: 930.3475935828877
           Else (feature 2 not in {4.0,5.0,6.0,7.0})
            Predict: 1204.5658914728683
         Else (feature 6 > 62.849999999999994)
          If (feature 0 in {1.0})
           If (feature 6 <= 63.55)
            Predict: 740.2142857142857
           Else (feature 6 > 63.55)
            Predict: 868.0508474576271
          Else (feature 0 not in {1.0})
           If (feature 7 <= 54.05)
            Predict: 1110.8
           Else (feature 7 > 54.05)
            Predict: 908.9833333333333
       Else (feature 1 > 4.845)
        If (feature 3 <= 5.075)
         If (feature 7 <= 56.05)
          If (feature 5 in {4.0,5.0,6.0})
           If (feature 6 <= 61.95)
            Predict: 890.8367346938776
           Else (feature 6 > 61.95)
            Predict: 971.3913043478261
          Else (feature 5 not in {4.0,5.0,6.0})
           If (feature 3 <= 4.845)
            Predict: 1113.2121212121212
           Else (feature 3 > 4.845)
            Predict: 1372.3863636363637
         Else (feature 7 > 56.05)
          If (feature 7 <= 60.2)
           If (feature 2 in {6.0,7.0})
            Predict: 890.2954545454545
           Else (feature 2 not in {6.0,7.0})
            Predict: 1152.0
          Else (feature 7 > 60.2)
           If (feature 2 in {6.0,7.0})
            Predict: 837.8928571428571
           Else (feature 2 not in {6.0,7.0})
            Predict: 1047.0
        Else (feature 3 > 5.075)
         If (feature 5 in {1.0,2.0,3.0,4.0,6.0})
          If (feature 2 in {4.0,5.0,6.0,7.0})
           If (feature 7 <= 56.05)
            Predict: 1043.8333333333333
           Else (feature 7 > 56.05)
            Predict: 1173.8095238095239
          Else (feature 2 not in {4.0,5.0,6.0,7.0})
           If (feature 0 in {1.0,2.0})
            Predict: 1793.0
           Else (feature 0 not in {1.0,2.0})
            Predict: 2102.0
         Else (feature 5 not in {1.0,2.0,3.0,4.0,6.0})
          If (feature 7 <= 57.05)
           If (feature 6 <= 59.849999999999994)
            Predict: 2329.0
           Else (feature 6 > 59.849999999999994)
            Predict: 2677.0
          Else (feature 7 > 57.05)
           If (feature 0 in {2.0})
            Predict: 1670.8
           Else (feature 0 not in {2.0})
            Predict: 2139.0
      Else (feature 4 > 0.495)
       If (feature 3 <= 5.335)
        If (feature 5 in {5.0,6.0})
         If (feature 7 <= 59.1)
          If (feature 7 <= 56.05)
           Predict: 900.0
          Else (feature 7 > 56.05)
           If (feature 6 <= 60.45)
            Predict: 956.0
           Else (feature 6 > 60.45)
            Predict: 947.0
         Else (feature 7 > 59.1)
          If (feature 7 <= 62.25)
           If (feature 0 in {1.0})
            Predict: 1193.0
           Else (feature 0 not in {1.0})
            Predict: 1275.5
          Else (feature 7 > 62.25)
           Predict: 1047.0
        Else (feature 5 not in {5.0,6.0})
         If (feature 0 in {0.0,1.0})
          If (feature 8 <= 2.995)
           If (feature 5 in {2.0,4.0})
            Predict: 1305.5
           Else (feature 5 not in {2.0,4.0})
            Predict: 1920.0
          Else (feature 8 > 2.995)
           If (feature 5 in {0.0,2.0,3.0,4.0})
            Predict: 1252.1944444444443
           Else (feature 5 not in {0.0,2.0,3.0,4.0})
            Predict: 1435.7894736842106
         Else (feature 0 not in {0.0,1.0})
          If (feature 1 <= 5.0649999999999995)
           If (feature 2 in {5.0,6.0,7.0})
            Predict: 1255.2608695652175
           Else (feature 2 not in {5.0,6.0,7.0})
            Predict: 1878.0
          Else (feature 1 > 5.0649999999999995)
           If (feature 2 in {6.0,7.0})
            Predict: 1306.9083333333333
           Else (feature 2 not in {6.0,7.0})
            Predict: 1786.425806451613
       Else (feature 3 > 5.335)
        If (feature 7 <= 61.1)
         If (feature 7 <= 59.1)
          If (feature 7 <= 58.05)
           Predict: 1625.0
          Else (feature 7 > 58.05)
           Predict: 1784.0
         Else (feature 7 > 59.1)
          If (feature 5 in {2.0,5.0})
           If (feature 2 in {6.0})
            Predict: 1089.0
           Else (feature 2 not in {6.0})
            Predict: 1609.0
          Else (feature 5 not in {2.0,5.0})
           Predict: 2391.0
        Else (feature 7 > 61.1)
         If (feature 5 in {1.0,4.0})
          If (feature 0 in {1.0,2.0})
           If (feature 0 in {2.0})
            Predict: 1160.0
           Else (feature 0 not in {2.0})
            Predict: 1521.0
          Else (feature 0 not in {1.0,2.0})
           Predict: 2079.0
         Else (feature 5 not in {1.0,4.0})
          Predict: 3266.6666666666665
    Else (feature 8 > 3.135)
     If (feature 1 <= 5.235)
      If (feature 1 <= 5.0649999999999995)
       If (feature 2 in {4.0,5.0,6.0,7.0,8.0})
        If (feature 2 in {6.0,7.0,8.0})
         If (feature 3 <= 4.845)
          If (feature 2 in {8.0})
           If (feature 7 <= 54.05)
            Predict: 613.0
           Else (feature 7 > 54.05)
            Predict: 627.0
          Else (feature 2 not in {8.0})
           If (feature 3 <= 4.705)
            Predict: 804.6666666666666
           Else (feature 3 > 4.705)
            Predict: 961.0
         Else (feature 3 > 4.845)
          If (feature 2 in {7.0,8.0})
           If (feature 2 in {8.0})
            Predict: 727.0
           Else (feature 2 not in {8.0})
            Predict: 1089.3333333333333
          Else (feature 2 not in {7.0,8.0})
           If (feature 6 <= 61.95)
            Predict: 841.0
           Else (feature 6 > 61.95)
            Predict: 1241.2282608695652
        Else (feature 2 not in {6.0,7.0,8.0})
         If (feature 0 in {0.0})
          If (feature 2 in {4.0})
           If (feature 7 <= 61.1)
            Predict: 1164.4285714285713
           Else (feature 7 > 61.1)
            Predict: 1257.0
          Else (feature 2 not in {4.0})
           If (feature 5 in {1.0,2.0,3.0,4.0,6.0})
            Predict: 1168.888888888889
           Else (feature 5 not in {1.0,2.0,3.0,4.0,6.0})
            Predict: 1668.0
         Else (feature 0 not in {0.0})
          If (feature 5 in {2.0,3.0,4.0,6.0})
           If (feature 4 <= 0.495)
            Predict: 1270.0
           Else (feature 4 > 0.495)
            Predict: 1431.787234042553
          Else (feature 5 not in {2.0,3.0,4.0,6.0})
           If (feature 0 in {1.0,2.0})
            Predict: 1582.7
           Else (feature 0 not in {1.0,2.0})
            Predict: 1724.5
       Else (feature 2 not in {4.0,5.0,6.0,7.0,8.0})
        If (feature 7 <= 58.05)
         If (feature 6 <= 62.25)
          If (feature 5 in {3.0})
           Predict: 1838.0
          Else (feature 5 not in {3.0})
           If (feature 5 in {2.0})
            Predict: 2074.125
           Else (feature 5 not in {2.0})
            Predict: 2199.0
         Else (feature 6 > 62.25)
          If (feature 4 <= 0.495)
           If (feature 8 <= 3.185)
            Predict: 1600.5
           Else (feature 8 > 3.185)
            Predict: 1705.0
          Else (feature 4 > 0.495)
           If (feature 7 <= 55.05)
            Predict: 1890.25
           Else (feature 7 > 55.05)
            Predict: 1788.0333333333333
        Else (feature 7 > 58.05)
         If (feature 7 <= 60.05)
          If (feature 8 <= 3.185)
           If (feature 5 in {1.0})
            Predict: 2105.0
           Else (feature 5 not in {1.0})
            Predict: 3378.0
          Else (feature 8 > 3.185)
           Predict: 3920.0
         Else (feature 7 > 60.05)
          If (feature 0 in {2.0})
           Predict: 1577.0
          Else (feature 0 not in {2.0})
           If (feature 4 <= 0.495)
            Predict: 1690.0
           Else (feature 4 > 0.495)
            Predict: 1678.0
      Else (feature 1 > 5.0649999999999995)
       If (feature 0 in {0.0,1.0,2.0,3.0})
        If (feature 5 in {4.0,5.0,6.0})
         If (feature 5 in {5.0,6.0})
          If (feature 6 <= 63.05)
           If (feature 6 <= 62.95)
            Predict: 1110.1190476190477
           Else (feature 6 > 62.95)
            Predict: 1367.6666666666667
          Else (feature 6 > 63.05)
           If (feature 2 in {4.0,7.0})
            Predict: 989.5555555555555
           Else (feature 2 not in {4.0,7.0})
            Predict: 1080.5555555555557
         Else (feature 5 not in {5.0,6.0})
          If (feature 6 <= 62.650000000000006)
           If (feature 6 <= 61.25)
            Predict: 1499.8333333333333
           Else (feature 6 > 61.25)
            Predict: 1348.5625
          Else (feature 6 > 62.650000000000006)
           If (feature 2 in {6.0,7.0})
            Predict: 1123.2142857142858
           Else (feature 2 not in {6.0,7.0})
            Predict: 1407.75
        Else (feature 5 not in {4.0,5.0,6.0})
         If (feature 6 <= 62.150000000000006)
          If (feature 3 <= 5.335)
           If (feature 2 in {6.0,7.0,8.0})
            Predict: 1329.4128440366972
           Else (feature 2 not in {6.0,7.0,8.0})
            Predict: 1829.1016042780748
          Else (feature 3 > 5.335)
           Predict: 2005.0
         Else (feature 6 > 62.150000000000006)
          If (feature 6 <= 63.55)
           If (feature 4 <= 0.515)
            Predict: 1569.5581395348838
           Else (feature 4 > 0.515)
            Predict: 1517.8416666666667
          Else (feature 6 > 63.55)
           If (feature 7 <= 61.1)
            Predict: 1418.7424242424242
           Else (feature 7 > 61.1)
            Predict: 2579.0
       Else (feature 0 not in {0.0,1.0,2.0,3.0})
        If (feature 2 in {4.0,5.0,6.0,7.0,8.0})
         If (feature 2 in {6.0,7.0,8.0})
          If (feature 7 <= 57.05)
           If (feature 5 in {2.0,3.0,4.0,5.0,6.0})
            Predict: 1229.5955882352941
           Else (feature 5 not in {2.0,3.0,4.0,5.0,6.0})
            Predict: 1462.4330708661416
          Else (feature 7 > 57.05)
           If (feature 4 <= 0.515)
            Predict: 1094.0
           Else (feature 4 > 0.515)
            Predict: 1173.6666666666667
         Else (feature 2 not in {6.0,7.0,8.0})
          If (feature 6 <= 61.849999999999994)
           If (feature 2 in {5.0})
            Predict: 1673.3503649635036
           Else (feature 2 not in {5.0})
            Predict: 1846.967032967033
          Else (feature 6 > 61.849999999999994)
           If (feature 6 <= 62.05)
            Predict: 1637.987341772152
           Else (feature 6 > 62.05)
            Predict: 1727.4155844155844
        Else (feature 2 not in {4.0,5.0,6.0,7.0,8.0})
         If (feature 3 <= 5.155)
          If (feature 1 <= 5.155)
           If (feature 2 in {2.0,3.0})
            Predict: 1975.3478260869565
           Else (feature 2 not in {2.0,3.0})
            Predict: 2186.6666666666665
          Else (feature 1 > 5.155)
           If (feature 4 <= 0.515)
            Predict: 2743.0
           Else (feature 4 > 0.515)
            Predict: 2012.0
         Else (feature 3 > 5.155)
          If (feature 2 in {3.0})
           If (feature 5 in {2.0,3.0,4.0,5.0})
            Predict: 2003.5
           Else (feature 5 not in {2.0,3.0,4.0,5.0})
            Predict: 2487.222222222222
          Else (feature 2 not in {3.0})
           If (feature 5 in {3.0,4.0,5.0})
            Predict: 1956.8793103448277
           Else (feature 5 not in {3.0,4.0,5.0})
            Predict: 2971.1935483870966
     Else (feature 1 > 5.235)
      If (feature 3 <= 5.335)
       If (feature 2 in {6.0,7.0,8.0})
        If (feature 2 in {7.0,8.0})
         If (feature 4 <= 0.705)
          If (feature 5 in {4.0,5.0,6.0})
           If (feature 3 <= 5.235)
            Predict: 994.2
           Else (feature 3 > 5.235)
            Predict: 1057.8333333333333
          Else (feature 5 not in {4.0,5.0,6.0})
           If (feature 8 <= 3.185)
            Predict: 1071.857142857143
           Else (feature 8 > 3.185)
            Predict: 1261.1375
         Else (feature 4 > 0.705)
          Predict: 2301.0
        Else (feature 2 not in {7.0,8.0})
         If (feature 5 in {4.0,5.0,6.0})
          If (feature 3 <= 5.155)
           Predict: 1077.0
          Else (feature 3 > 5.155)
           If (feature 8 <= 3.185)
            Predict: 1185.3333333333333
           Else (feature 8 > 3.185)
            Predict: 1237.159090909091
         Else (feature 5 not in {4.0,5.0,6.0})
          If (feature 7 <= 61.1)
           If (feature 5 in {2.0,3.0})
            Predict: 1481.9142857142858
           Else (feature 5 not in {2.0,3.0})
            Predict: 1685.012987012987
          Else (feature 7 > 61.1)
           If (feature 0 in {1.0})
            Predict: 1847.0
           Else (feature 0 not in {1.0})
            Predict: 2362.0
       Else (feature 2 not in {6.0,7.0,8.0})
        If (feature 8 <= 3.225)
         If (feature 6 <= 61.349999999999994)
          If (feature 2 in {2.0,4.0,5.0})
           If (feature 2 in {4.0,5.0})
            Predict: 1813.1517857142858
           Else (feature 2 not in {4.0,5.0})
            Predict: 1960.05
          Else (feature 2 not in {2.0,4.0,5.0})
           If (feature 2 in {1.0})
            Predict: 2032.3333333333333
           Else (feature 2 not in {1.0})
            Predict: 2171.714285714286
         Else (feature 6 > 61.349999999999994)
          If (feature 7 <= 57.05)
           If (feature 6 <= 61.45)
            Predict: 1714.7142857142858
           Else (feature 6 > 61.45)
            Predict: 1667.0
          Else (feature 7 > 57.05)
           If (feature 7 <= 58.05)
            Predict: 1579.0
           Else (feature 7 > 58.05)
            Predict: 1600.0
        Else (feature 8 > 3.225)
         If (feature 2 in {4.0,5.0})
          If (feature 5 in {4.0,5.0,6.0})
           If (feature 5 in {5.0,6.0})
            Predict: 1312.5333333333333
           Else (feature 5 not in {5.0,6.0})
            Predict: 1619.0625
          Else (feature 5 not in {4.0,5.0,6.0})
           If (feature 0 in {1.0,2.0})
            Predict: 1725.7435897435898
           Else (feature 0 not in {1.0,2.0})
            Predict: 1942.8714285714286
         Else (feature 2 not in {4.0,5.0})
          If (feature 2 in {2.0,3.0})
           If (feature 6 <= 61.650000000000006)
            Predict: 2353.575
           Else (feature 6 > 61.650000000000006)
            Predict: 2146.1666666666665
          Else (feature 2 not in {2.0,3.0})
           If (feature 5 in {2.0,3.0,4.0,6.0})
            Predict: 2183.6666666666665
           Else (feature 5 not in {2.0,3.0,4.0,6.0})
            Predict: 4519.0
      Else (feature 3 > 5.335)
       If (feature 5 in {1.0,2.0,3.0,4.0,5.0,6.0})
        If (feature 2 in {5.0,6.0,7.0,8.0})
         If (feature 5 in {4.0,5.0,6.0})
          If (feature 0 in {2.0,3.0,4.0})
           If (feature 7 <= 62.25)
            Predict: 1314.764705882353
           Else (feature 7 > 62.25)
            Predict: 1626.25
          Else (feature 0 not in {2.0,3.0,4.0})
           If (feature 7 <= 60.05)
            Predict: 1748.2
           Else (feature 7 > 60.05)
            Predict: 1014.0
         Else (feature 5 not in {4.0,5.0,6.0})
          If (feature 2 in {6.0,7.0,8.0})
           If (feature 2 in {7.0,8.0})
            Predict: 1404.1375
           Else (feature 2 not in {7.0,8.0})
            Predict: 1630.3937007874015
          Else (feature 2 not in {6.0,7.0,8.0})
           If (feature 4 <= 0.635)
            Predict: 1821.909090909091
           Else (feature 4 > 0.635)
            Predict: 2271.1
        Else (feature 2 not in {5.0,6.0,7.0,8.0})
         If (feature 0 in {1.0,3.0})
          If (feature 2 in {4.0})
           If (feature 6 <= 61.05)
            Predict: 1790.2666666666667
           Else (feature 6 > 61.05)
            Predict: 2017.2222222222222
          Else (feature 2 not in {4.0})
           If (feature 2 in {2.0,3.0})
            Predict: 2122.785714285714
           Else (feature 2 not in {2.0,3.0})
            Predict: 2753.0
         Else (feature 0 not in {1.0,3.0})
          If (feature 2 in {4.0})
           If (feature 8 <= 3.395)
            Predict: 1920.2674418604652
           Else (feature 8 > 3.395)
            Predict: 2283.2727272727275
          Else (feature 2 not in {4.0})
           If (feature 5 in {4.0,5.0,6.0})
            Predict: 1971.2857142857142
           Else (feature 5 not in {4.0,5.0,6.0})
            Predict: 2466.968503937008
       Else (feature 5 not in {1.0,2.0,3.0,4.0,5.0,6.0})
        If (feature 2 in {3.0,4.0,5.0,6.0,7.0})
         If (feature 2 in {5.0,6.0,7.0})
          If (feature 6 <= 62.95)
           If (feature 7 <= 58.45)
            Predict: 1870.276595744681
           Else (feature 7 > 58.45)
            Predict: 1755.1764705882354
          Else (feature 6 > 62.95)
           If (feature 6 <= 64.05)
            Predict: 2044.125
           Else (feature 6 > 64.05)
            Predict: 2204.2
         Else (feature 2 not in {5.0,6.0,7.0})
          If (feature 6 <= 62.05)
           If (feature 6 <= 60.75)
            Predict: 2543.777777777778
           Else (feature 6 > 60.75)
            Predict: 2245.2
          Else (feature 6 > 62.05)
           If (feature 0 in {2.0})
            Predict: 2233.6
           Else (feature 0 not in {2.0})
            Predict: 2940.6
        Else (feature 2 not in {3.0,4.0,5.0,6.0,7.0})
         If (feature 2 in {2.0})
          If (feature 6 <= 60.45)
           Predict: 2945.0
          Else (feature 6 > 60.45)
           Predict: 3999.6875
         Else (feature 2 not in {2.0})
          Predict: 6568.333333333333
   Else (feature 3 > 5.525)
    If (feature 3 <= 5.995)
     If (feature 2 in {6.0,7.0,8.0})
      If (feature 2 in {8.0})
       If (feature 3 <= 5.765)
        If (feature 0 in {0.0,1.0,2.0,3.0})
         If (feature 8 <= 3.395)
          Predict: 933.0
         Else (feature 8 > 3.395)
          If (feature 6 <= 63.25)
           If (feature 5 in {2.0,5.0})
            Predict: 1183.0
           Else (feature 5 not in {2.0,5.0})
            Predict: 1351.5
          Else (feature 6 > 63.25)
           If (feature 1 <= 5.835)
            Predict: 1118.4545454545455
           Else (feature 1 > 5.835)
            Predict: 1238.0
        Else (feature 0 not in {0.0,1.0,2.0,3.0})
         Predict: 1761.0
       Else (feature 3 > 5.765)
        If (feature 1 <= 5.745)
         Predict: 1298.0
        Else (feature 1 > 5.745)
         If (feature 8 <= 3.955)
          If (feature 4 <= 0.745)
           Predict: 1865.0
          Else (feature 4 > 0.745)
           If (feature 1 <= 6.005)
            Predict: 2310.0
           Else (feature 1 > 6.005)
            Predict: 1939.0
         Else (feature 8 > 3.955)
          If (feature 1 <= 6.005)
           Predict: 1786.0
          Else (feature 1 > 6.005)
           If (feature 3 <= 5.845)
            Predict: 1789.0
           Else (feature 3 > 5.845)
            Predict: 1725.5
      Else (feature 2 not in {8.0})
       If (feature 4 <= 0.715)
        If (feature 2 in {7.0})
         If (feature 0 in {1.0,2.0,3.0})
          If (feature 5 in {2.0,3.0,4.0,5.0,6.0})
           If (feature 5 in {5.0,6.0})
            Predict: 1829.076923076923
           Else (feature 5 not in {5.0,6.0})
            Predict: 2024.3305084745762
          Else (feature 5 not in {2.0,3.0,4.0,5.0,6.0})
           If (feature 1 <= 5.535)
            Predict: 1664.0
           Else (feature 1 > 5.535)
            Predict: 2233.7011494252874
         Else (feature 0 not in {1.0,2.0,3.0})
          If (feature 1 <= 5.535)
           If (feature 7 <= 57.05)
            Predict: 1345.0
           Else (feature 7 > 57.05)
            Predict: 2035.0
          Else (feature 1 > 5.535)
           If (feature 4 <= 0.635)
            Predict: 1065.0
           Else (feature 4 > 0.635)
            Predict: 2304.3555555555554
        Else (feature 2 not in {7.0})
         If (feature 0 in {0.0,1.0,2.0,3.0})
          If (feature 5 in {3.0,4.0,5.0,6.0})
           If (feature 4 <= 0.635)
            Predict: 1258.0
           Else (feature 4 > 0.635)
            Predict: 2054.0295857988167
          Else (feature 5 not in {3.0,4.0,5.0,6.0})
           If (feature 3 <= 5.695)
            Predict: 2374.1716417910447
           Else (feature 3 > 5.695)
            Predict: 2513.097014925373
         Else (feature 0 not in {0.0,1.0,2.0,3.0})
          If (feature 5 in {2.0,3.0,4.0,5.0,6.0})
           If (feature 1 <= 5.535)
            Predict: 1628.0
           Else (feature 1 > 5.535)
            Predict: 2378.992700729927
          Else (feature 5 not in {2.0,3.0,4.0,5.0,6.0})
           If (feature 1 <= 5.535)
            Predict: 1816.0
           Else (feature 1 > 5.535)
            Predict: 2798.8837209302324
       Else (feature 4 > 0.715)
        If (feature 2 in {7.0})
         If (feature 4 <= 0.825)
          If (feature 3 <= 5.765)
           If (feature 5 in {3.0,4.0,5.0,6.0})
            Predict: 1985.3095238095239
           Else (feature 5 not in {3.0,4.0,5.0,6.0})
            Predict: 2460.0
          Else (feature 3 > 5.765)
           If (feature 0 in {0.0,2.0,3.0})
            Predict: 2345.3885714285716
           Else (feature 0 not in {0.0,2.0,3.0})
            Predict: 2513.856115107914
         Else (feature 4 > 0.825)
          If (feature 3 <= 5.845)
           If (feature 7 <= 59.1)
            Predict: 2322.25
           Else (feature 7 > 59.1)
            Predict: 2672.0
          Else (feature 3 > 5.845)
           If (feature 1 <= 6.005)
            Predict: 2527.5882352941176
           Else (feature 1 > 6.005)
            Predict: 3003.5
        Else (feature 2 not in {7.0})
         If (feature 7 <= 61.1)
          If (feature 3 <= 5.845)
           If (feature 5 in {5.0,6.0})
            Predict: 2093.6923076923076
           Else (feature 5 not in {5.0,6.0})
            Predict: 2676.572093023256
          Else (feature 3 > 5.845)
           If (feature 4 <= 0.745)
            Predict: 2686.232558139535
           Else (feature 4 > 0.745)
            Predict: 2940.6425339366515
         Else (feature 7 > 61.1)
          If (feature 5 in {1.0,2.0,3.0,4.0,6.0})
           If (feature 8 <= 3.5549999999999997)
            Predict: 2042.8
           Else (feature 8 > 3.5549999999999997)
            Predict: 2320.6
          Else (feature 5 not in {1.0,2.0,3.0,4.0,6.0})
           Predict: 3046.0
     Else (feature 2 not in {6.0,7.0,8.0})
      If (feature 5 in {4.0,5.0,6.0})
       If (feature 4 <= 0.745)
        If (feature 5 in {6.0})
         If (feature 4 <= 0.635)
          If (feature 8 <= 3.395)
           Predict: 1415.0
          Else (feature 8 > 3.395)
           Predict: 949.0
         Else (feature 4 > 0.635)
          If (feature 2 in {4.0,5.0})
           If (feature 6 <= 62.849999999999994)
            Predict: 1983.4
           Else (feature 6 > 62.849999999999994)
            Predict: 1875.2666666666667
          Else (feature 2 not in {4.0,5.0})
           If (feature 6 <= 61.05)
            Predict: 2126.5
           Else (feature 6 > 61.05)
            Predict: 2326.2
        Else (feature 5 not in {6.0})
         If (feature 3 <= 5.695)
          If (feature 4 <= 0.635)
           If (feature 6 <= 59.349999999999994)
            Predict: 1394.0
           Else (feature 6 > 59.349999999999994)
            Predict: 1933.923076923077
          Else (feature 4 > 0.635)
           If (feature 6 <= 61.849999999999994)
            Predict: 2189.714285714286
           Else (feature 6 > 61.849999999999994)
            Predict: 2335.7272727272725
         Else (feature 3 > 5.695)
          If (feature 0 in {0.0,1.0,2.0,3.0})
           If (feature 7 <= 62.25)
            Predict: 2429.3984962406016
           Else (feature 7 > 62.25)
            Predict: 2112.8888888888887
          Else (feature 0 not in {0.0,1.0,2.0,3.0})
           If (feature 1 <= 5.6850000000000005)
            Predict: 2484.8214285714284
           Else (feature 1 > 5.6850000000000005)
            Predict: 2825.958620689655
       Else (feature 4 > 0.745)
        If (feature 8 <= 3.605)
         If (feature 7 <= 59.1)
          If (feature 5 in {6.0})
           If (feature 2 in {5.0})
            Predict: 2052.0
           Else (feature 2 not in {5.0})
            Predict: 2262.0
          Else (feature 5 not in {6.0})
           If (feature 5 in {5.0})
            Predict: 2584.0
           Else (feature 5 not in {5.0})
            Predict: 2828.8
         Else (feature 7 > 59.1)
          If (feature 7 <= 60.05)
           Predict: 2840.0
          Else (feature 7 > 60.05)
           If (feature 6 <= 59.849999999999994)
            Predict: 2530.0
           Else (feature 6 > 59.849999999999994)
            Predict: 2616.0
        Else (feature 8 > 3.605)
         If (feature 8 <= 3.885)
          If (feature 5 in {5.0,6.0})
           If (feature 6 <= 61.05)
            Predict: 2922.4
           Else (feature 6 > 61.05)
            Predict: 2635.4285714285716
          Else (feature 5 not in {5.0,6.0})
           If (feature 6 <= 63.25)
            Predict: 3211.3414634146343
           Else (feature 6 > 63.25)
            Predict: 2751.0
         Else (feature 8 > 3.885)
          If (feature 5 in {5.0})
           If (feature 7 <= 56.05)
            Predict: 3074.0
           Else (feature 7 > 56.05)
            Predict: 3351.75
          Else (feature 5 not in {5.0})
           If (feature 7 <= 56.05)
            Predict: 3658.0
           Else (feature 7 > 56.05)
            Predict: 3500.0
      Else (feature 5 not in {4.0,5.0,6.0})
       If (feature 2 in {4.0,5.0})
        If (feature 8 <= 3.605)
         If (feature 0 in {0.0,1.0,2.0,3.0})
          If (feature 4 <= 0.635)
           If (feature 0 in {1.0})
            Predict: 1758.6666666666667
           Else (feature 0 not in {1.0})
            Predict: 2118.875
          Else (feature 4 > 0.635)
           If (feature 4 <= 0.705)
            Predict: 2723.3696498054474
           Else (feature 4 > 0.705)
            Predict: 2888.1056603773586
         Else (feature 0 not in {0.0,1.0,2.0,3.0})
          If (feature 6 <= 61.95)
           If (feature 4 <= 0.635)
            Predict: 2426.5454545454545
           Else (feature 4 > 0.635)
            Predict: 3262.09375
          Else (feature 6 > 61.95)
           If (feature 5 in {1.0,2.0,3.0})
            Predict: 2878.680412371134
           Else (feature 5 not in {1.0,2.0,3.0})
            Predict: 3301.576923076923
        Else (feature 8 > 3.605)
         If (feature 1 <= 5.745)
          If (feature 5 in {1.0,2.0,3.0})
           If (feature 6 <= 62.75)
            Predict: 3081.3333333333335
           Else (feature 6 > 62.75)
            Predict: 2756.56
          Else (feature 5 not in {1.0,2.0,3.0})
           If (feature 1 <= 5.6850000000000005)
            Predict: 2943.75
           Else (feature 1 > 5.6850000000000005)
            Predict: 4289.0
         Else (feature 1 > 5.745)
          If (feature 2 in {5.0})
           If (feature 1 <= 5.835)
            Predict: 3092.0714285714284
           Else (feature 1 > 5.835)
            Predict: 3435.4337349397592
          Else (feature 2 not in {5.0})
           If (feature 6 <= 61.349999999999994)
            Predict: 3273.625
           Else (feature 6 > 61.349999999999994)
            Predict: 3685.2833333333333
       Else (feature 2 not in {4.0,5.0})
        If (feature 4 <= 0.745)
         If (feature 2 in {1.0,3.0})
          If (feature 5 in {3.0})
           If (feature 0 in {3.0})
            Predict: 2800.0666666666666
           Else (feature 0 not in {3.0})
            Predict: 3093.8333333333335
          Else (feature 5 not in {3.0})
           If (feature 3 <= 5.695)
            Predict: 3238.7619047619046
           Else (feature 3 > 5.695)
            Predict: 3808.923076923077
         Else (feature 2 not in {1.0,3.0})
          If (feature 4 <= 0.705)
           If (feature 5 in {2.0,3.0})
            Predict: 3419.962962962963
           Else (feature 5 not in {2.0,3.0})
            Predict: 4046.769230769231
          Else (feature 4 > 0.705)
           If (feature 8 <= 3.5549999999999997)
            Predict: 4189.521739130435
           Else (feature 8 > 3.5549999999999997)
            Predict: 3653.25
        Else (feature 4 > 0.745)
         If (feature 1 <= 5.835)
          If (feature 2 in {2.0,3.0})
           If (feature 5 in {2.0,3.0})
            Predict: 3586.3846153846152
           Else (feature 5 not in {2.0,3.0})
            Predict: 5423.0
          Else (feature 2 not in {2.0,3.0})
           Predict: 4798.0
         Else (feature 1 > 5.835)
          If (feature 0 in {2.0,3.0})
           If (feature 2 in {2.0,3.0})
            Predict: 4153.4375
           Else (feature 2 not in {2.0,3.0})
            Predict: 4879.5
          Else (feature 0 not in {2.0,3.0})
           If (feature 7 <= 56.05)
            Predict: 5074.727272727273
           Else (feature 7 > 56.05)
            Predict: 4386.285714285715
    Else (feature 3 > 5.995)
     If (feature 8 <= 3.705)
      If (feature 4 <= 0.905)
       If (feature 2 in {3.0,4.0,5.0,6.0,7.0,8.0})
        If (feature 2 in {7.0,8.0})
         If (feature 5 in {3.0,4.0,5.0,6.0})
          If (feature 2 in {8.0})
           If (feature 7 <= 62.25)
            Predict: 1648.25
           Else (feature 7 > 62.25)
            Predict: 1334.0
          Else (feature 2 not in {8.0})
           If (feature 1 <= 6.295)
            Predict: 2353.6666666666665
           Else (feature 1 > 6.295)
            Predict: 3371.0
         Else (feature 5 not in {3.0,4.0,5.0,6.0})
          If (feature 1 <= 6.165)
           If (feature 2 in {8.0})
            Predict: 1749.0
           Else (feature 2 not in {8.0})
            Predict: 2707.4242424242425
          Else (feature 1 > 6.165)
           If (feature 2 in {8.0})
            Predict: 2170.0
           Else (feature 2 not in {8.0})
            Predict: 3331.6363636363635
        Else (feature 2 not in {7.0,8.0})
         If (feature 1 <= 6.165)
          If (feature 0 in {1.0,2.0,3.0})
           If (feature 5 in {3.0,4.0,5.0,6.0})
            Predict: 2666.9117647058824
           Else (feature 5 not in {3.0,4.0,5.0,6.0})
            Predict: 3535.8
          Else (feature 0 not in {1.0,2.0,3.0})
           If (feature 1 <= 5.235)
            Predict: 2030.0
           Else (feature 1 > 5.235)
            Predict: 3602.56
         Else (feature 1 > 6.165)
          If (feature 5 in {4.0,5.0,6.0})
           If (feature 4 <= 0.825)
            Predict: 2664.0
           Else (feature 4 > 0.825)
            Predict: 3283.4285714285716
          Else (feature 5 not in {4.0,5.0,6.0})
           If (feature 7 <= 61.1)
            Predict: 4338.444444444444
           Else (feature 7 > 61.1)
            Predict: 3899.2
       Else (feature 2 not in {3.0,4.0,5.0,6.0,7.0,8.0})
        If (feature 1 <= 6.005)
         Predict: 6674.0
        Else (feature 1 > 6.005)
         Predict: 7889.0
      Else (feature 4 > 0.905)
       If (feature 2 in {4.0,7.0,8.0})
        If (feature 7 <= 62.25)
         If (feature 0 in {1.0,3.0})
          If (feature 1 <= 6.395)
           Predict: 3379.0
          Else (feature 1 > 6.395)
           Predict: 3558.6666666666665
         Else (feature 0 not in {1.0,3.0})
          Predict: 3924.0
        Else (feature 7 > 62.25)
         Predict: 2838.5
       Else (feature 2 not in {4.0,7.0,8.0})
        If (feature 5 in {5.0,6.0})
         If (feature 3 <= 6.385)
          Predict: 2700.0
         Else (feature 3 > 6.385)
          Predict: 2812.0
        Else (feature 5 not in {5.0,6.0})
         If (feature 0 in {0.0})
          If (feature 5 in {2.0,4.0})
           If (feature 2 in {2.0,6.0})
            Predict: 4089.0
           Else (feature 2 not in {2.0,6.0})
            Predict: 4474.0
          Else (feature 5 not in {2.0,4.0})
           Predict: 5078.0
         Else (feature 0 not in {0.0})
          Predict: 5394.0
     Else (feature 8 > 3.705)
      If (feature 5 in {5.0,6.0})
       If (feature 0 in {1.0})
        If (feature 3 <= 6.385)
         If (feature 1 <= 6.005)
          Predict: 2676.0
         Else (feature 1 > 6.005)
          If (feature 6 <= 62.25)
           If (feature 4 <= 0.905)
            Predict: 2618.5
           Else (feature 4 > 0.905)
            Predict: 3052.0
          Else (feature 6 > 62.25)
           If (feature 7 <= 55.05)
            Predict: 2764.3333333333335
           Else (feature 7 > 55.05)
            Predict: 3244.4666666666667
        Else (feature 3 > 6.385)
         Predict: 3946.0
       Else (feature 0 not in {1.0})
        If (feature 2 in {6.0,7.0,8.0})
         If (feature 2 in {8.0})
          If (feature 6 <= 61.75)
           If (feature 1 <= 6.395)
            Predict: 2195.0
           Else (feature 1 > 6.395)
            Predict: 2064.0
          Else (feature 6 > 61.75)
           Predict: 2363.0
         Else (feature 2 not in {8.0})
          If (feature 5 in {6.0})
           If (feature 4 <= 0.905)
            Predict: 2728.4
           Else (feature 4 > 0.905)
            Predict: 3097.6666666666665
          Else (feature 5 not in {6.0})
           If (feature 3 <= 6.385)
            Predict: 3213.5882352941176
           Else (feature 3 > 6.385)
            Predict: 3873.0
        Else (feature 2 not in {6.0,7.0,8.0})
         If (feature 1 <= 6.165)
          If (feature 4 <= 0.825)
           If (feature 7 <= 55.05)
            Predict: 3355.0
           Else (feature 7 > 55.05)
            Predict: 2676.6666666666665
          Else (feature 4 > 0.825)
           If (feature 1 <= 6.005)
            Predict: 4089.0
           Else (feature 1 > 6.005)
            Predict: 3376.810344827586
         Else (feature 1 > 6.165)
          If (feature 5 in {6.0})
           If (feature 2 in {5.0})
            Predict: 3178.4285714285716
           Else (feature 2 not in {5.0})
            Predict: 3551.125
          Else (feature 5 not in {6.0})
           If (feature 3 <= 6.165)
            Predict: 3630.4
           Else (feature 3 > 6.165)
            Predict: 3821.0
      Else (feature 5 not in {5.0,6.0})
       If (feature 3 <= 6.165)
        If (feature 0 in {0.0})
         If (feature 4 <= 0.905)
          If (feature 6 <= 64.05)
           Predict: 4796.0
          Else (feature 6 > 64.05)
           If (feature 7 <= 56.05)
            Predict: 3397.4285714285716
           Else (feature 7 > 56.05)
            Predict: 3265.3333333333335
         Else (feature 4 > 0.905)
          If (feature 6 <= 62.95)
           Predict: 1917.0
          Else (feature 6 > 62.95)
           If (feature 2 in {6.0,7.0,8.0})
            Predict: 2970.0833333333335
           Else (feature 2 not in {6.0,7.0,8.0})
            Predict: 4909.666666666667
        Else (feature 0 not in {0.0})
         If (feature 0 in {1.0,4.0})
          If (feature 2 in {6.0,7.0,8.0})
           If (feature 2 in {7.0,8.0})
            Predict: 3102.742424242424
           Else (feature 2 not in {7.0,8.0})
            Predict: 3672.3949579831933
          Else (feature 2 not in {6.0,7.0,8.0})
           If (feature 2 in {3.0,4.0,5.0})
            Predict: 4277.914893617021
           Else (feature 2 not in {3.0,4.0,5.0})
            Predict: 5262.083333333333
         Else (feature 0 not in {1.0,4.0})
          If (feature 4 <= 0.825)
           Predict: 3093.0
          Else (feature 4 > 0.825)
           If (feature 7 <= 55.05)
            Predict: 3735.3571428571427
           Else (feature 7 > 55.05)
            Predict: 4036.2256097560976
       Else (feature 3 > 6.165)
        If (feature 0 in {0.0,1.0,2.0,3.0})
         If (feature 2 in {7.0,8.0})
          If (feature 8 <= 3.825)
           If (feature 2 in {8.0})
            Predict: 2077.3333333333335
           Else (feature 2 not in {8.0})
            Predict: 3465.546391752577
          Else (feature 8 > 3.825)
           If (feature 5 in {2.0,3.0,4.0})
            Predict: 3654.0169491525426
           Else (feature 5 not in {2.0,3.0,4.0})
            Predict: 3970.6111111111113
         Else (feature 2 not in {7.0,8.0})
          If (feature 7 <= 62.25)
           If (feature 2 in {6.0})
            Predict: 4154.3785310734465
           Else (feature 2 not in {6.0})
            Predict: 4677.2797619047615
          Else (feature 7 > 62.25)
           If (feature 5 in {0.0,1.0,3.0})
            Predict: 3961.777777777778
           Else (feature 5 not in {0.0,1.0,3.0})
            Predict: 4127.0
        Else (feature 0 not in {0.0,1.0,2.0,3.0})
         If (feature 2 in {4.0,5.0,6.0,7.0,8.0})
          If (feature 2 in {7.0,8.0})
           If (feature 2 in {8.0})
            Predict: 2822.75
           Else (feature 2 not in {8.0})
            Predict: 3917.9493670886077
          Else (feature 2 not in {7.0,8.0})
           If (feature 3 <= 6.525)
            Predict: 4675.491935483871
           Else (feature 3 > 6.525)
            Predict: 2075.0
         Else (feature 2 not in {4.0,5.0,6.0,7.0,8.0})
          If (feature 2 in {1.0,3.0})
           If (feature 7 <= 54.05)
            Predict: 7854.4
           Else (feature 7 > 54.05)
            Predict: 5489.5
          Else (feature 2 not in {1.0,3.0})
           If (feature 8 <= 3.885)
            Predict: 7467.1
           Else (feature 8 > 3.885)
            Predict: 6488.0
  Else (feature 4 > 0.995)
   If (feature 8 <= 4.365)
    If (feature 4 <= 1.165)
     If (feature 2 in {4.0,5.0,6.0,7.0,8.0})
      If (feature 6 <= 63.25)
       If (feature 7 <= 58.05)
        If (feature 5 in {4.0,5.0,6.0})
         If (feature 2 in {7.0,8.0})
          If (feature 4 <= 1.085)
           If (feature 4 <= 1.0150000000000001)
            Predict: 3625.109090909091
           Else (feature 4 > 1.0150000000000001)
            Predict: 3963.264957264957
          Else (feature 4 > 1.085)
           If (feature 2 in {8.0})
            Predict: 3703.0
           Else (feature 2 not in {8.0})
            Predict: 4571.52
         Else (feature 2 not in {7.0,8.0})
          If (feature 4 <= 1.085)
           If (feature 5 in {5.0,6.0})
            Predict: 4383.55
           Else (feature 5 not in {5.0,6.0})
            Predict: 5066.955947136564
          Else (feature 4 > 1.085)
           If (feature 5 in {5.0,6.0})
            Predict: 4986.139240506329
           Else (feature 5 not in {5.0,6.0})
            Predict: 5539.585034013606
        Else (feature 5 not in {4.0,5.0,6.0})
         If (feature 2 in {6.0,7.0,8.0})
          If (feature 2 in {7.0,8.0})
           If (feature 5 in {2.0,3.0})
            Predict: 4356.652317880795
           Else (feature 5 not in {2.0,3.0})
            Predict: 4818.061452513966
          Else (feature 2 not in {7.0,8.0})
           If (feature 4 <= 1.085)
            Predict: 5294.630372492837
           Else (feature 4 > 1.085)
            Predict: 5975.184210526316
         Else (feature 2 not in {6.0,7.0,8.0})
          If (feature 5 in {3.0})
           If (feature 0 in {0.0,1.0,2.0,3.0})
            Predict: 6257.948387096774
           Else (feature 0 not in {0.0,1.0,2.0,3.0})
            Predict: 7002.611374407583
          Else (feature 5 not in {3.0})
           If (feature 1 <= 6.625)
            Predict: 7235.390879478828
           Else (feature 1 > 6.625)
            Predict: 8419.850746268658
       Else (feature 7 > 58.05)
        If (feature 5 in {4.0,5.0,6.0})
         If (feature 2 in {7.0,8.0})
          If (feature 8 <= 4.075)
           If (feature 2 in {8.0})
            Predict: 2896.722222222222
           Else (feature 2 not in {8.0})
            Predict: 3885.7327586206898
          Else (feature 8 > 4.075)
           If (feature 2 in {8.0})
            Predict: 3421.75
           Else (feature 2 not in {8.0})
            Predict: 4627.92
         Else (feature 2 not in {7.0,8.0})
          If (feature 5 in {5.0,6.0})
           If (feature 8 <= 4.035)
            Predict: 4215.6736842105265
           Else (feature 8 > 4.035)
            Predict: 4806.969696969697
          Else (feature 5 not in {5.0,6.0})
           If (feature 4 <= 1.085)
            Predict: 4936.245283018868
           Else (feature 4 > 1.085)
            Predict: 5475.129032258064
        Else (feature 5 not in {4.0,5.0,6.0})
         If (feature 5 in {2.0,3.0})
          If (feature 3 <= 6.295)
           If (feature 1 <= 6.395)
            Predict: 4513.4
           Else (feature 1 > 6.395)
            Predict: 2108.0
          Else (feature 3 > 6.295)
           If (feature 0 in {0.0,1.0,3.0})
            Predict: 5120.33574879227
           Else (feature 0 not in {0.0,1.0,3.0})
            Predict: 5505.275862068966
         Else (feature 5 not in {2.0,3.0})
          If (feature 1 <= 6.465)
           If (feature 1 <= 6.395)
            Predict: 5421.897727272727
           Else (feature 1 > 6.395)
            Predict: 5746.944444444444
          Else (feature 1 > 6.465)
           If (feature 2 in {6.0,7.0,8.0})
            Predict: 4799.771844660194
           Else (feature 2 not in {6.0,7.0,8.0})
            Predict: 7130.64705882353
      Else (feature 6 > 63.25)
       If (feature 5 in {4.0,5.0,6.0})
        If (feature 0 in {0.0})
         If (feature 7 <= 60.05)
          If (feature 5 in {5.0,6.0})
           If (feature 7 <= 54.05)
            Predict: 3880.0
           Else (feature 7 > 54.05)
            Predict: 3123.3529411764707
          Else (feature 5 not in {5.0,6.0})
           If (feature 3 <= 6.165)
            Predict: 3339.5
           Else (feature 3 > 6.165)
            Predict: 4118.681818181818
         Else (feature 7 > 60.05)
          Predict: 2958.25
        Else (feature 0 not in {0.0})
         If (feature 1 <= 6.465)
          If (feature 5 in {6.0})
           If (feature 0 in {1.0})
            Predict: 3562.5
           Else (feature 0 not in {1.0})
            Predict: 4018.5
          Else (feature 5 not in {6.0})
           If (feature 7 <= 54.05)
            Predict: 4613.125
           Else (feature 7 > 54.05)
            Predict: 4168.277777777777
         Else (feature 1 > 6.465)
          If (feature 7 <= 58.05)
           If (feature 4 <= 1.085)
            Predict: 5225.0
           Else (feature 4 > 1.085)
            Predict: 4876.333333333333
          Else (feature 7 > 58.05)
           If (feature 5 in {6.0})
            Predict: 3958.5714285714284
           Else (feature 5 not in {6.0})
            Predict: 4537.25
       Else (feature 5 not in {4.0,5.0,6.0})
        If (feature 0 in {0.0})
         If (feature 2 in {6.0,7.0,8.0})
          If (feature 5 in {1.0,2.0,3.0})
           If (feature 2 in {8.0})
            Predict: 2347.4285714285716
           Else (feature 2 not in {8.0})
            Predict: 3887.7462686567164
          Else (feature 5 not in {1.0,2.0,3.0})
           If (feature 4 <= 1.0150000000000001)
            Predict: 4938.625
           Else (feature 4 > 1.0150000000000001)
            Predict: 4406.6
         Else (feature 2 not in {6.0,7.0,8.0})
          If (feature 1 <= 6.395)
           If (feature 7 <= 56.05)
            Predict: 4984.333333333333
           Else (feature 7 > 56.05)
            Predict: 5749.272727272727
          Else (feature 1 > 6.395)
           Predict: 7335.0
        Else (feature 0 not in {0.0})
         If (feature 1 <= 6.535)
          If (feature 2 in {6.0,7.0,8.0})
           If (feature 2 in {7.0,8.0})
            Predict: 3952.6792452830186
           Else (feature 2 not in {7.0,8.0})
            Predict: 5038.622807017544
          Else (feature 2 not in {6.0,7.0,8.0})
           If (feature 5 in {3.0})
            Predict: 6076.457142857143
           Else (feature 5 not in {3.0})
            Predict: 6533.285714285715
         Else (feature 1 > 6.535)
          If (feature 5 in {3.0})
           If (feature 0 in {1.0})
            Predict: 3424.75
           Else (feature 0 not in {1.0})
            Predict: 4066.0
          Else (feature 5 not in {3.0})
           If (feature 3 <= 6.525)
            Predict: 4742.0
           Else (feature 3 > 6.525)
            Predict: 4939.0
     Else (feature 2 not in {4.0,5.0,6.0,7.0,8.0})
      If (feature 2 in {2.0,3.0})
       If (feature 1 <= 6.535)
        If (feature 6 <= 62.95)
         If (feature 6 <= 62.75)
          If (feature 6 <= 62.650000000000006)
           If (feature 4 <= 1.0150000000000001)
            Predict: 8475.415254237289
           Else (feature 4 > 1.0150000000000001)
            Predict: 8903.481927710844
          Else (feature 6 > 62.650000000000006)
           If (feature 0 in {3.0})
            Predict: 6473.5
           Else (feature 0 not in {3.0})
            Predict: 8401.0
         Else (feature 6 > 62.75)
          If (feature 7 <= 56.05)
           If (feature 8 <= 4.035)
            Predict: 7520.0
           Else (feature 8 > 4.035)
            Predict: 7738.0
          Else (feature 7 > 56.05)
           If (feature 4 <= 1.0150000000000001)
            Predict: 10597.111111111111
           Else (feature 4 > 1.0150000000000001)
            Predict: 9705.0
        Else (feature 6 > 62.95)
         If (feature 1 <= 6.295)
          If (feature 5 in {2.0,3.0,4.0})
           If (feature 5 in {4.0})
            Predict: 4312.0
           Else (feature 5 not in {4.0})
            Predict: 6662.4
          Else (feature 5 not in {2.0,3.0,4.0})
           If (feature 0 in {1.0})
            Predict: 8359.0
           Else (feature 0 not in {1.0})
            Predict: 8622.5
         Else (feature 1 > 6.295)
          If (feature 5 in {3.0,4.0,6.0})
           If (feature 5 in {6.0})
            Predict: 4640.0
           Else (feature 5 not in {6.0})
            Predict: 7033.68
          Else (feature 5 not in {3.0,4.0,6.0})
           If (feature 0 in {0.0,1.0,3.0})
            Predict: 8176.25
           Else (feature 0 not in {0.0,1.0,3.0})
            Predict: 9309.266666666666
       Else (feature 1 > 6.535)
        If (feature 8 <= 3.885)
         If (feature 5 in {4.0,5.0})
          If (feature 3 <= 6.525)
           Predict: 5684.0
          Else (feature 3 > 6.525)
           If (feature 4 <= 1.005)
            Predict: 4032.0
           Else (feature 4 > 1.005)
            Predict: 5171.0
         Else (feature 5 not in {4.0,5.0})
          If (feature 4 <= 1.0150000000000001)
           If (feature 0 in {3.0})
            Predict: 6670.0
           Else (feature 0 not in {3.0})
            Predict: 7725.0
          Else (feature 4 > 1.0150000000000001)
           Predict: 9254.0
        Else (feature 8 > 3.885)
         If (feature 0 in {2.0,3.0})
          If (feature 5 in {3.0,4.0})
           If (feature 5 in {4.0})
            Predict: 6177.0
           Else (feature 5 not in {4.0})
            Predict: 8280.814814814816
          Else (feature 5 not in {3.0,4.0})
           If (feature 8 <= 4.155)
            Predict: 10098.146341463415
           Else (feature 8 > 4.155)
            Predict: 13405.0
         Else (feature 0 not in {2.0,3.0})
          If (feature 3 <= 6.625)
           If (feature 5 in {3.0,4.0,5.0})
            Predict: 8183.0
           Else (feature 5 not in {3.0,4.0,5.0})
            Predict: 10173.192307692309
          Else (feature 3 > 6.625)
           If (feature 5 in {2.0,3.0,4.0,5.0})
            Predict: 9348.19642857143
           Else (feature 5 not in {2.0,3.0,4.0,5.0})
            Predict: 12701.375
      Else (feature 2 not in {2.0,3.0})
       If (feature 5 in {2.0,3.0,4.0,6.0})
        If (feature 5 in {3.0,4.0,6.0})
         If (feature 5 in {4.0,6.0})
          If (feature 5 in {6.0})
           Predict: 5361.0
          Else (feature 5 not in {6.0})
           Predict: 6694.0
         Else (feature 5 not in {4.0,6.0})
          If (feature 1 <= 6.535)
           If (feature 8 <= 3.955)
            Predict: 7720.222222222223
           Else (feature 8 > 3.955)
            Predict: 8475.0
          Else (feature 1 > 6.535)
           If (feature 7 <= 59.1)
            Predict: 9543.21052631579
           Else (feature 7 > 59.1)
            Predict: 9012.333333333334
        Else (feature 5 not in {3.0,4.0,6.0})
         If (feature 1 <= 6.465)
          If (feature 6 <= 62.05)
           If (feature 0 in {2.0})
            Predict: 10346.0
           Else (feature 0 not in {2.0})
            Predict: 10688.0
          Else (feature 6 > 62.05)
           Predict: 9248.0
         Else (feature 1 > 6.465)
          If (feature 7 <= 56.05)
           If (feature 4 <= 1.085)
            Predict: 12302.333333333334
           Else (feature 4 > 1.085)
            Predict: 11631.0
          Else (feature 7 > 56.05)
           If (feature 7 <= 57.05)
            Predict: 9627.0
           Else (feature 7 > 57.05)
            Predict: 11019.0
       Else (feature 5 not in {2.0,3.0,4.0,6.0})
        If (feature 5 in {1.0})
         If (feature 1 <= 6.465)
          If (feature 6 <= 62.25)
           Predict: 12030.0
          Else (feature 6 > 62.25)
           If (feature 7 <= 55.05)
            Predict: 11214.0
           Else (feature 7 > 55.05)
            Predict: 11495.5
         Else (feature 1 > 6.465)
          If (feature 8 <= 4.075)
           If (feature 6 <= 59.849999999999994)
            Predict: 11288.333333333334
           Else (feature 6 > 59.849999999999994)
            Predict: 13067.333333333334
          Else (feature 8 > 4.075)
           Predict: 15464.0
        Else (feature 5 not in {1.0})
         If (feature 1 <= 6.535)
          If (feature 6 <= 61.650000000000006)
           If (feature 6 <= 61.55)
            Predict: 15300.5
           Else (feature 6 > 61.55)
            Predict: 16234.0
          Else (feature 6 > 61.650000000000006)
           Predict: 15150.0
         Else (feature 1 > 6.535)
          If (feature 1 <= 6.625)
           If (feature 4 <= 1.0350000000000001)
            Predict: 17590.0
           Else (feature 4 > 1.0350000000000001)
            Predict: 15813.0
          Else (feature 1 > 6.625)
           If (feature 0 in {1.0})
            Predict: 17499.0
           Else (feature 0 not in {1.0})
            Predict: 18279.0
    Else (feature 4 > 1.165)
     If (feature 2 in {5.0,6.0,7.0,8.0})
      If (feature 3 <= 7.525)
       If (feature 6 <= 63.05)
        If (feature 8 <= 4.245)
         If (feature 5 in {5.0,6.0})
          If (feature 5 in {6.0})
           If (feature 8 <= 4.155)
            Predict: 4300.9
           Else (feature 8 > 4.155)
            Predict: 4908.640625
          Else (feature 5 not in {6.0})
           If (feature 2 in {7.0,8.0})
            Predict: 4752.666666666667
           Else (feature 2 not in {7.0,8.0})
            Predict: 5578.756302521008
         Else (feature 5 not in {5.0,6.0})
          If (feature 0 in {0.0,3.0})
           If (feature 2 in {6.0,7.0,8.0})
            Predict: 5570.711656441717
           Else (feature 2 not in {6.0,7.0,8.0})
            Predict: 7825.078125
          Else (feature 0 not in {0.0,3.0})
           If (feature 8 <= 4.155)
            Predict: 6292.482758620689
           Else (feature 8 > 4.155)
            Predict: 6892.74609375
        Else (feature 8 > 4.245)
         If (feature 5 in {4.0,5.0,6.0})
          If (feature 6 <= 60.45)
           If (feature 0 in {1.0,3.0})
            Predict: 6138.35294117647
           Else (feature 0 not in {1.0,3.0})
            Predict: 7546.8125
          Else (feature 6 > 60.45)
           If (feature 0 in {2.0,3.0})
            Predict: 5755.886075949367
           Else (feature 0 not in {2.0,3.0})
            Predict: 6094.6
         Else (feature 5 not in {4.0,5.0,6.0})
          If (feature 2 in {6.0,7.0,8.0})
           If (feature 4 <= 1.355)
            Predict: 6543.658730158731
           Else (feature 4 > 1.355)
            Predict: 7607.142857142857
          Else (feature 2 not in {6.0,7.0,8.0})
           If (feature 3 <= 7.075)
            Predict: 8894.162162162162
           Else (feature 3 > 7.075)
            Predict: 10813.0
       Else (feature 6 > 63.05)
        If (feature 2 in {7.0,8.0})
         If (feature 2 in {8.0})
          If (feature 5 in {5.0})
           Predict: 2380.0
          Else (feature 5 not in {5.0})
           If (feature 8 <= 4.245)
            Predict: 3743.25
           Else (feature 8 > 4.245)
            Predict: 4066.6
         Else (feature 2 not in {8.0})
          If (feature 6 <= 64.05)
           If (feature 3 <= 6.745)
            Predict: 4587.411764705882
           Else (feature 3 > 6.745)
            Predict: 5022.588235294118
          Else (feature 6 > 64.05)
           If (feature 7 <= 56.05)
            Predict: 4879.0
           Else (feature 7 > 56.05)
            Predict: 4348.8
        Else (feature 2 not in {7.0,8.0})
         If (feature 6 <= 63.55)
          If (feature 2 in {6.0})
           If (feature 8 <= 4.245)
            Predict: 6034.4
           Else (feature 8 > 4.245)
            Predict: 5532.84
          Else (feature 2 not in {6.0})
           If (feature 1 <= 6.755)
            Predict: 7831.636363636364
           Else (feature 1 > 6.755)
            Predict: 5881.833333333333
         Else (feature 6 > 63.55)
          If (feature 5 in {5.0,6.0})
           If (feature 7 <= 55.05)
            Predict: 4054.0
           Else (feature 7 > 55.05)
            Predict: 5073.15
          Else (feature 5 not in {5.0,6.0})
           If (feature 5 in {4.0})
            Predict: 5929.625
           Else (feature 5 not in {4.0})
            Predict: 6928.333333333333
      Else (feature 3 > 7.525)
       If (feature 0 in {0.0})
        If (feature 7 <= 58.05)
         Predict: 11489.0
        Else (feature 7 > 58.05)
         Predict: 9369.0
       Else (feature 0 not in {0.0})
        Predict: 18207.0
     Else (feature 2 not in {5.0,6.0,7.0,8.0})
      If (feature 5 in {4.0,5.0,6.0})
       If (feature 2 in {3.0,4.0})
        If (feature 5 in {5.0,6.0})
         If (feature 1 <= 6.885)
          If (feature 0 in {1.0,2.0,3.0})
           If (feature 2 in {3.0})
            Predict: 5376.0
           Else (feature 2 not in {3.0})
            Predict: 5687.321428571428
          Else (feature 0 not in {1.0,2.0,3.0})
           If (feature 7 <= 56.25)
            Predict: 5901.1
           Else (feature 7 > 56.25)
            Predict: 6682.333333333333
         Else (feature 1 > 6.885)
          If (feature 4 <= 1.355)
           If (feature 0 in {1.0,2.0,3.0})
            Predict: 5958.9
           Else (feature 0 not in {1.0,2.0,3.0})
            Predict: 7242.1578947368425
          Else (feature 4 > 1.355)
           If (feature 0 in {0.0})
            Predict: 6727.0
           Else (feature 0 not in {0.0})
            Predict: 9828.0
        Else (feature 5 not in {5.0,6.0})
         If (feature 0 in {0.0,1.0,2.0,3.0})
          If (feature 3 <= 6.875)
           If (feature 2 in {3.0})
            Predict: 5174.0
           Else (feature 2 not in {3.0})
            Predict: 6990.580645161291
          Else (feature 3 > 6.875)
           If (feature 1 <= 7.095)
            Predict: 7558.538461538462
           Else (feature 1 > 7.095)
            Predict: 8606.666666666666
         Else (feature 0 not in {0.0,1.0,2.0,3.0})
          If (feature 6 <= 62.650000000000006)
           If (feature 7 <= 56.05)
            Predict: 8153.307692307692
           Else (feature 7 > 56.05)
            Predict: 7764.875
          Else (feature 6 > 62.650000000000006)
           Predict: 5055.0
       Else (feature 2 not in {3.0,4.0})
        If (feature 5 in {5.0,6.0})
         If (feature 8 <= 4.245)
          If (feature 6 <= 61.349999999999994)
           If (feature 8 <= 4.155)
            Predict: 6378.0
           Else (feature 8 > 4.155)
            Predict: 5893.0
          Else (feature 6 > 61.349999999999994)
           If (feature 6 <= 61.55)
            Predict: 7367.0
           Else (feature 6 > 61.55)
            Predict: 7083.0
         Else (feature 8 > 4.245)
          If (feature 4 <= 1.225)
           Predict: 6867.0
          Else (feature 4 > 1.225)
           If (feature 0 in {3.0})
            Predict: 7690.5
           Else (feature 0 not in {3.0})
            Predict: 7709.5
        Else (feature 5 not in {5.0,6.0})
         If (feature 2 in {1.0})
          If (feature 0 in {3.0})
           Predict: 8294.0
          Else (feature 0 not in {3.0})
           If (feature 7 <= 53.95)
            Predict: 8266.0
           Else (feature 7 > 53.95)
            Predict: 8431.0
         Else (feature 2 not in {1.0})
          If (feature 1 <= 6.885)
           Predict: 9604.0
          Else (feature 1 > 6.885)
           If (feature 8 <= 4.245)
            Predict: 8920.0
           Else (feature 8 > 4.245)
            Predict: 9232.0
      Else (feature 5 not in {4.0,5.0,6.0})
       If (feature 3 <= 6.875)
        If (feature 7 <= 57.05)
         If (feature 1 <= 6.755)
          If (feature 8 <= 4.245)
           If (feature 0 in {2.0,3.0})
            Predict: 8954.6
           Else (feature 0 not in {2.0,3.0})
            Predict: 9692.6
          Else (feature 8 > 4.245)
           If (feature 7 <= 56.05)
            Predict: 11268.0
           Else (feature 7 > 56.05)
            Predict: 9586.0
         Else (feature 1 > 6.755)
          If (feature 4 <= 1.225)
           If (feature 5 in {3.0})
            Predict: 9503.078947368422
           Else (feature 5 not in {3.0})
            Predict: 11147.272727272728
          Else (feature 4 > 1.225)
           If (feature 6 <= 62.25)
            Predict: 9772.142857142857
           Else (feature 6 > 62.25)
            Predict: 12386.666666666666
        Else (feature 7 > 57.05)
         If (feature 2 in {1.0,4.0})
          If (feature 5 in {3.0})
           If (feature 3 <= 6.745)
            Predict: 7602.0
           Else (feature 3 > 6.745)
            Predict: 8526.526315789473
          Else (feature 5 not in {3.0})
           If (feature 0 in {3.0})
            Predict: 9705.0
           Else (feature 0 not in {3.0})
            Predict: 10367.0
         Else (feature 2 not in {1.0,4.0})
          If (feature 6 <= 61.650000000000006)
           If (feature 5 in {3.0})
            Predict: 9210.57142857143
           Else (feature 5 not in {3.0})
            Predict: 11370.111111111111
          Else (feature 6 > 61.650000000000006)
           If (feature 3 <= 6.745)
            Predict: 9550.57142857143
           Else (feature 3 > 6.745)
            Predict: 9149.666666666666
       Else (feature 3 > 6.875)
        If (feature 2 in {4.0})
         If (feature 5 in {2.0,3.0})
          If (feature 4 <= 1.225)
           If (feature 7 <= 57.05)
            Predict: 9448.0
           Else (feature 7 > 57.05)
            Predict: 8827.142857142857
          Else (feature 4 > 1.225)
           If (feature 6 <= 61.55)
            Predict: 9767.367346938776
           Else (feature 6 > 61.55)
            Predict: 10436.5
         Else (feature 5 not in {2.0,3.0})
          If (feature 4 <= 1.225)
           If (feature 0 in {2.0,3.0})
            Predict: 10102.4
           Else (feature 0 not in {2.0,3.0})
            Predict: 10821.75
          Else (feature 4 > 1.225)
           If (feature 8 <= 4.245)
            Predict: 10388.0
           Else (feature 8 > 4.245)
            Predict: 12217.181818181818
        Else (feature 2 not in {4.0})
         If (feature 8 <= 4.155)
          If (feature 1 <= 6.885)
           If (feature 2 in {3.0})
            Predict: 11430.0
           Else (feature 2 not in {3.0})
            Predict: 15185.0
          Else (feature 1 > 6.885)
           If (feature 4 <= 1.225)
            Predict: 10126.7
           Else (feature 4 > 1.225)
            Predict: 9324.0
         Else (feature 8 > 4.155)
          If (feature 0 in {2.0,3.0,4.0})
           If (feature 6 <= 61.55)
            Predict: 11516.863636363636
           Else (feature 6 > 61.55)
            Predict: 12443.423076923076
          Else (feature 0 not in {2.0,3.0,4.0})
           Predict: 15420.0
   Else (feature 8 > 4.365)
    If (feature 4 <= 2.005)
     If (feature 5 in {4.0,5.0,6.0})
      If (feature 4 <= 1.5950000000000002)
       If (feature 5 in {6.0})
        If (feature 2 in {5.0,6.0,7.0,8.0})
         If (feature 3 <= 7.075)
          If (feature 3 <= 6.875)
           If (feature 2 in {7.0,8.0})
            Predict: 4413.0
           Else (feature 2 not in {7.0,8.0})
            Predict: 5488.0
          Else (feature 3 > 6.875)
           If (feature 7 <= 58.05)
            Predict: 5465.272727272727
           Else (feature 7 > 58.05)
            Predict: 5709.666666666667
         Else (feature 3 > 7.075)
          If (feature 2 in {7.0,8.0})
           If (feature 0 in {0.0,2.0})
            Predict: 5805.875
           Else (feature 0 not in {0.0,2.0})
            Predict: 6288.190476190476
          Else (feature 2 not in {7.0,8.0})
           If (feature 0 in {0.0,1.0,3.0})
            Predict: 7092.25
           Else (feature 0 not in {0.0,1.0,3.0})
            Predict: 7444.190476190476
        Else (feature 2 not in {5.0,6.0,7.0,8.0})
         If (feature 6 <= 62.45)
          If (feature 3 <= 7.335)
           If (feature 0 in {2.0})
            Predict: 5986.666666666667
           Else (feature 0 not in {2.0})
            Predict: 8316.555555555555
          Else (feature 3 > 7.335)
           If (feature 0 in {1.0,3.0})
            Predict: 8621.0
           Else (feature 0 not in {1.0,3.0})
            Predict: 9145.142857142857
         Else (feature 6 > 62.45)
          If (feature 1 <= 7.095)
           If (feature 7 <= 57.05)
            Predict: 5808.0
           Else (feature 7 > 57.05)
            Predict: 6449.0
          Else (feature 1 > 7.095)
           If (feature 0 in {0.0})
            Predict: 7186.0
           Else (feature 0 not in {0.0})
            Predict: 8636.666666666666
       Else (feature 5 not in {6.0})
        If (feature 3 <= 7.335)
         If (feature 6 <= 64.05)
          If (feature 8 <= 4.545)
           If (feature 2 in {7.0,8.0})
            Predict: 6300.473684210527
           Else (feature 2 not in {7.0,8.0})
            Predict: 8337.182608695652
          Else (feature 8 > 4.545)
           If (feature 2 in {6.0,7.0,8.0})
            Predict: 7768.575757575758
           Else (feature 2 not in {6.0,7.0,8.0})
            Predict: 10401.58208955224
         Else (feature 6 > 64.05)
          If (feature 3 <= 7.075)
           If (feature 4 <= 1.355)
            Predict: 4734.75
           Else (feature 4 > 1.355)
            Predict: 6106.1578947368425
          Else (feature 3 > 7.075)
           If (feature 2 in {8.0})
            Predict: 4065.5
           Else (feature 2 not in {8.0})
            Predict: 8081.3125
        Else (feature 3 > 7.335)
         If (feature 2 in {7.0,8.0})
          If (feature 4 <= 1.5150000000000001)
           If (feature 6 <= 61.95)
            Predict: 7009.571428571428
           Else (feature 6 > 61.95)
            Predict: 8371.0
          Else (feature 4 > 1.5150000000000001)
           If (feature 6 <= 62.75)
            Predict: 7841.907407407408
           Else (feature 6 > 62.75)
            Predict: 6592.666666666667
         Else (feature 2 not in {7.0,8.0})
          If (feature 7 <= 58.05)
           If (feature 4 <= 1.5150000000000001)
            Predict: 10307.785714285714
           Else (feature 4 > 1.5150000000000001)
            Predict: 10750.382978723404
          Else (feature 7 > 58.05)
           If (feature 4 <= 1.5150000000000001)
            Predict: 9905.829268292682
           Else (feature 4 > 1.5150000000000001)
            Predict: 10351.0
      Else (feature 4 > 1.5950000000000002)
       If (feature 3 <= 8.015)
        If (feature 3 <= 7.525)
         If (feature 3 <= 7.335)
          If (feature 0 in {0.0})
           Predict: 5240.0
          Else (feature 0 not in {0.0})
           Predict: 6223.0
         Else (feature 3 > 7.335)
          If (feature 6 <= 62.75)
           If (feature 2 in {7.0})
            Predict: 8458.57142857143
           Else (feature 2 not in {7.0})
            Predict: 10407.478260869566
          Else (feature 6 > 62.75)
           If (feature 7 <= 58.05)
            Predict: 8144.0
           Else (feature 7 > 58.05)
            Predict: 9787.4
        Else (feature 3 > 7.525)
         If (feature 7 <= 54.95)
          If (feature 0 in {2.0})
           Predict: 9090.0
          Else (feature 0 not in {2.0})
           If (feature 6 <= 62.45)
            Predict: 9890.555555555555
           Else (feature 6 > 62.45)
            Predict: 7068.0
         Else (feature 7 > 54.95)
          If (feature 2 in {1.0,7.0,8.0})
           If (feature 8 <= 4.975)
            Predict: 8839.36923076923
           Else (feature 8 > 4.975)
            Predict: 10308.533333333333
          Else (feature 2 not in {1.0,7.0,8.0})
           If (feature 6 <= 62.25)
            Predict: 11621.505
           Else (feature 6 > 62.25)
            Predict: 12993.80487804878
       Else (feature 3 > 8.015)
        If (feature 5 in {6.0})
         If (feature 0 in {0.0})
          Predict: 8362.0
         Else (feature 0 not in {0.0})
          If (feature 2 in {6.0,7.0})
           If (feature 0 in {2.0,4.0})
            Predict: 11398.25
           Else (feature 0 not in {2.0,4.0})
            Predict: 11634.666666666666
          Else (feature 2 not in {6.0,7.0})
           If (feature 1 <= 8.055)
            Predict: 14403.0
           Else (feature 1 > 8.055)
            Predict: 13521.0
        Else (feature 5 not in {6.0})
         If (feature 6 <= 60.45)
          If (feature 7 <= 59.1)
           If (feature 2 in {8.0})
            Predict: 5045.0
           Else (feature 2 not in {8.0})
            Predict: 13575.833333333334
          Else (feature 7 > 59.1)
           If (feature 7 <= 62.25)
            Predict: 15591.818181818182
           Else (feature 7 > 62.25)
            Predict: 11262.0
         Else (feature 6 > 60.45)
          If (feature 2 in {6.0,7.0})
           If (feature 0 in {2.0,3.0,4.0})
            Predict: 14405.5
           Else (feature 0 not in {2.0,3.0,4.0})
            Predict: 17600.0
          Else (feature 2 not in {6.0,7.0})
           If (feature 2 in {5.0})
            Predict: 16941.6
           Else (feature 2 not in {5.0})
            Predict: 17849.0
     Else (feature 5 not in {4.0,5.0,6.0})
      If (feature 1 <= 7.345)
       If (feature 2 in {6.0,7.0,8.0})
        If (feature 2 in {7.0,8.0})
         If (feature 0 in {0.0})
          If (feature 1 <= 5.745)
           Predict: 1262.0
          Else (feature 1 > 5.745)
           If (feature 6 <= 61.849999999999994)
            Predict: 8287.0
           Else (feature 6 > 61.849999999999994)
            Predict: 5826.428571428572
         Else (feature 0 not in {0.0})
          If (feature 3 <= 7.075)
           If (feature 6 <= 62.95)
            Predict: 6267.416666666667
           Else (feature 6 > 62.95)
            Predict: 4607.0
          Else (feature 3 > 7.075)
           If (feature 4 <= 1.355)
            Predict: 6510.6
           Else (feature 4 > 1.355)
            Predict: 7924.68085106383
        Else (feature 2 not in {7.0,8.0})
         If (feature 5 in {3.0})
          If (feature 7 <= 55.05)
           If (feature 8 <= 4.545)
            Predict: 7639.0
           Else (feature 8 > 4.545)
            Predict: 9724.333333333334
          Else (feature 7 > 55.05)
           If (feature 4 <= 1.355)
            Predict: 7147.0
           Else (feature 4 > 1.355)
            Predict: 9562.2
         Else (feature 5 not in {3.0})
          If (feature 4 <= 1.355)
           If (feature 0 in {2.0})
            Predict: 6801.2
           Else (feature 0 not in {2.0})
            Predict: 7625.0
          Else (feature 4 > 1.355)
           If (feature 7 <= 56.05)
            Predict: 10628.869565217392
           Else (feature 7 > 56.05)
            Predict: 11272.380952380952
       Else (feature 2 not in {6.0,7.0,8.0})
        If (feature 3 <= 7.075)
         If (feature 5 in {2.0})
          If (feature 4 <= 1.355)
           Predict: 7983.0
          Else (feature 4 > 1.355)
           If (feature 0 in {0.0})
            Predict: 8216.0
           Else (feature 0 not in {0.0})
            Predict: 9542.0
         Else (feature 5 not in {2.0})
          If (feature 7 <= 56.05)
           If (feature 2 in {4.0})
            Predict: 9301.75
           Else (feature 2 not in {4.0})
            Predict: 10398.57142857143
          Else (feature 7 > 56.05)
           If (feature 2 in {5.0})
            Predict: 9471.0
           Else (feature 2 not in {5.0})
            Predict: 11471.333333333334
        Else (feature 3 > 7.075)
         If (feature 8 <= 4.545)
          If (feature 5 in {3.0})
           If (feature 3 <= 7.335)
            Predict: 11286.432432432432
           Else (feature 3 > 7.335)
            Predict: 12746.111111111111
          Else (feature 5 not in {3.0})
           If (feature 5 in {2.0})
            Predict: 13206.421052631578
           Else (feature 5 not in {2.0})
            Predict: 14465.423076923076
         Else (feature 8 > 4.545)
          If (feature 2 in {5.0})
           If (feature 5 in {3.0})
            Predict: 12391.771428571428
           Else (feature 5 not in {3.0})
            Predict: 14205.02380952381
          Else (feature 2 not in {5.0})
           If (feature 5 in {3.0})
            Predict: 13666.365853658537
           Else (feature 5 not in {3.0})
            Predict: 16096.4375
      Else (feature 1 > 7.345)
       If (feature 2 in {6.0,7.0,8.0})
        If (feature 2 in {7.0,8.0})
         If (feature 8 <= 4.975)
          If (feature 3 <= 8.015)
           If (feature 6 <= 63.55)
            Predict: 9633.109375
           Else (feature 6 > 63.55)
            Predict: 7143.733333333334
          Else (feature 3 > 8.015)
           If (feature 0 in {1.0,3.0,4.0})
            Predict: 12761.333333333334
           Else (feature 0 not in {1.0,3.0,4.0})
            Predict: 15821.0
         Else (feature 8 > 4.975)
          If (feature 0 in {0.0})
           If (feature 7 <= 56.05)
            Predict: 13190.666666666666
           Else (feature 7 > 56.05)
            Predict: 6895.0
          Else (feature 0 not in {0.0})
           If (feature 7 <= 59.1)
            Predict: 16705.4
           Else (feature 7 > 59.1)
            Predict: 11502.75
        Else (feature 2 not in {7.0,8.0})
         If (feature 4 <= 1.5950000000000002)
          If (feature 5 in {3.0})
           If (feature 6 <= 61.150000000000006)
            Predict: 10596.166666666666
           Else (feature 6 > 61.150000000000006)
            Predict: 10119.363636363636
          Else (feature 5 not in {3.0})
           If (feature 7 <= 58.05)
            Predict: 12039.714285714286
           Else (feature 7 > 58.05)
            Predict: 12436.09756097561
         Else (feature 4 > 1.5950000000000002)
          If (feature 3 <= 8.015)
           If (feature 5 in {2.0,3.0})
            Predict: 12685.03125
           Else (feature 5 not in {2.0,3.0})
            Predict: 14211.222222222223
          Else (feature 3 > 8.015)
           If (feature 0 in {3.0})
            Predict: 18493.0
           Else (feature 0 not in {3.0})
            Predict: 18641.5
       Else (feature 2 not in {6.0,7.0,8.0})
        If (feature 3 <= 7.525)
         If (feature 5 in {2.0,3.0})
          If (feature 2 in {1.0,4.0,5.0})
           If (feature 8 <= 4.545)
            Predict: 13256.585714285715
           Else (feature 8 > 4.545)
            Predict: 13907.890410958904
          Else (feature 2 not in {1.0,4.0,5.0})
           If (feature 6 <= 61.55)
            Predict: 16777.555555555555
           Else (feature 6 > 61.55)
            Predict: 15801.083333333334
         Else (feature 5 not in {2.0,3.0})
          If (feature 2 in {5.0})
           If (feature 7 <= 54.05)
            Predict: 17223.0
           Else (feature 7 > 54.05)
            Predict: 15107.71875
          Else (feature 2 not in {5.0})
           If (feature 4 <= 1.5150000000000001)
            Predict: 16219.666666666666
           Else (feature 4 > 1.5150000000000001)
            Predict: 17042.875
        Else (feature 3 > 7.525)
         If (feature 4 <= 1.5950000000000002)
          If (feature 5 in {0.0,2.0,3.0})
           If (feature 2 in {5.0})
            Predict: 13215.07142857143
           Else (feature 2 not in {5.0})
            Predict: 14712.714285714286
          Else (feature 5 not in {0.0,2.0,3.0})
           If (feature 0 in {1.0,2.0})
            Predict: 15347.0
           Else (feature 0 not in {1.0,2.0})
            Predict: 17444.666666666668
         Else (feature 4 > 1.5950000000000002)
          If (feature 5 in {1.0,3.0})
           If (feature 8 <= 4.645)
            Predict: 15300.42857142857
           Else (feature 8 > 4.645)
            Predict: 16474.71875
          Else (feature 5 not in {1.0,3.0})
           If (feature 7 <= 55.95)
            Predict: 15777.0
           Else (feature 7 > 55.95)
            Predict: 17418.5
    Else (feature 4 > 2.005)
     If (feature 0 in {0.0})
      If (feature 7 <= 59.1)
       If (feature 5 in {1.0,2.0,3.0,4.0,5.0,6.0})
        If (feature 6 <= 64.05)
         Predict: 14799.0
        Else (feature 6 > 64.05)
         Predict: 10444.57627118644
       Else (feature 5 not in {1.0,2.0,3.0,4.0,5.0,6.0})
        If (feature 7 <= 53.95)
         Predict: 15964.0
        Else (feature 7 > 53.95)
         If (feature 7 <= 55.05)
          Predict: 16386.0
         Else (feature 7 > 55.05)
          Predict: 16086.0
      Else (feature 7 > 59.1)
       If (feature 5 in {2.0,3.0,5.0})
        If (feature 6 <= 58.650000000000006)
         Predict: 13387.0
        Else (feature 6 > 58.650000000000006)
         If (feature 5 in {5.0})
          Predict: 10671.0
         Else (feature 5 not in {5.0})
          Predict: 13721.0
       Else (feature 5 not in {2.0,3.0,5.0})
        If (feature 2 in {7.0})
         If (feature 8 <= 4.975)
          If (feature 7 <= 61.1)
           Predict: 14080.0
          Else (feature 7 > 61.1)
           If (feature 1 <= 8.055)
            Predict: 14948.0
           Else (feature 1 > 8.055)
            Predict: 15627.0
         Else (feature 8 > 4.975)
          Predict: 13587.0
        Else (feature 2 not in {7.0})
         Predict: 17561.333333333332
     Else (feature 0 not in {0.0})
      If (feature 6 <= 63.25)
       If (feature 0 in {1.0,3.0})
        If (feature 2 in {8.0})
         If (feature 0 in {1.0})
          If (feature 7 <= 58.05)
           Predict: 7026.0
          Else (feature 7 > 58.05)
           Predict: 10863.0
         Else (feature 0 not in {1.0})
          If (feature 5 in {3.0,4.0,6.0})
           If (feature 8 <= 4.975)
            Predict: 6899.0
           Else (feature 8 > 4.975)
            Predict: 8796.5
          Else (feature 5 not in {3.0,4.0,6.0})
           Predict: 16193.0
        Else (feature 2 not in {8.0})
         If (feature 3 <= 8.015)
          If (feature 2 in {7.0})
           If (feature 0 in {3.0})
            Predict: 13084.896551724138
           Else (feature 0 not in {3.0})
            Predict: 16186.555555555555
          Else (feature 2 not in {7.0})
           If (feature 2 in {6.0})
            Predict: 15591.692307692309
           Else (feature 2 not in {6.0})
            Predict: 15913.4
         Else (feature 3 > 8.015)
          If (feature 6 <= 62.650000000000006)
           If (feature 2 in {4.0,6.0,7.0})
            Predict: 15190.486631016043
           Else (feature 2 not in {4.0,6.0,7.0})
            Predict: 16608.46052631579
          Else (feature 6 > 62.650000000000006)
           If (feature 7 <= 58.05)
            Predict: 14971.384615384615
           Else (feature 7 > 58.05)
            Predict: 13568.92857142857
       Else (feature 0 not in {1.0,3.0})
        If (feature 8 <= 4.975)
         If (feature 5 in {0.0,5.0,6.0})
          If (feature 5 in {0.0,6.0})
           If (feature 1 <= 8.055)
            Predict: 12084.0
           Else (feature 1 > 8.055)
            Predict: 12940.307692307691
          Else (feature 5 not in {0.0,6.0})
           If (feature 3 <= 8.015)
            Predict: 9781.0
           Else (feature 3 > 8.015)
            Predict: 15643.434782608696
         Else (feature 5 not in {0.0,5.0,6.0})
          If (feature 2 in {7.0,8.0})
           If (feature 2 in {8.0})
            Predict: 9789.0
           Else (feature 2 not in {8.0})
            Predict: 15724.25
          Else (feature 2 not in {7.0,8.0})
           If (feature 3 <= 8.015)
            Predict: 16770.333333333332
           Else (feature 3 > 8.015)
            Predict: 17640.4
        Else (feature 8 > 4.975)
         If (feature 6 <= 62.55)
          If (feature 5 in {2.0,5.0,6.0})
           If (feature 2 in {5.0,6.0,7.0})
            Predict: 15246.496598639456
           Else (feature 2 not in {5.0,6.0,7.0})
            Predict: 16931.736842105263
          Else (feature 5 not in {2.0,5.0,6.0})
           If (feature 6 <= 60.75)
            Predict: 15371.285714285714
           Else (feature 6 > 60.75)
            Predict: 16920.611650485436
         Else (feature 6 > 62.55)
          If (feature 5 in {0.0,1.0,5.0,6.0})
           If (feature 2 in {5.0,6.0,7.0})
            Predict: 14046.775510204081
           Else (feature 2 not in {5.0,6.0,7.0})
            Predict: 16564.444444444445
          Else (feature 5 not in {0.0,1.0,5.0,6.0})
           If (feature 3 <= 8.015)
            Predict: 15291.8
           Else (feature 3 > 8.015)
            Predict: 16246.95652173913
      Else (feature 6 > 63.25)
       If (feature 3 <= 8.015)
        If (feature 2 in {8.0})
         Predict: 6855.0
        Else (feature 2 not in {8.0})
         If (feature 7 <= 57.05)
          If (feature 1 <= 8.055)
           If (feature 5 in {5.0,6.0})
            Predict: 11113.25
           Else (feature 5 not in {5.0,6.0})
            Predict: 15590.625
          Else (feature 1 > 8.055)
           If (feature 6 <= 63.55)
            Predict: 12829.0
           Else (feature 6 > 63.55)
            Predict: 15888.0
         Else (feature 7 > 57.05)
          If (feature 2 in {6.0,7.0})
           Predict: 12396.304347826086
          Else (feature 2 not in {6.0,7.0})
           If (feature 0 in {1.0})
            Predict: 11954.0
           Else (feature 0 not in {1.0})
            Predict: 15763.75
       Else (feature 3 > 8.015)
        If (feature 2 in {5.0,8.0})
         If (feature 7 <= 62.25)
          Predict: 12961.0
         Else (feature 7 > 62.25)
          Predict: 11548.0
        Else (feature 2 not in {5.0,8.0})
         If (feature 1 <= 8.055)
          If (feature 5 in {3.0})
           Predict: 16148.0
          Else (feature 5 not in {3.0})
           Predict: 17672.0
         Else (feature 1 > 8.055)
          If (feature 5 in {3.0,4.0,5.0,6.0})
           If (feature 5 in {3.0})
            Predict: 13007.0
           Else (feature 5 not in {3.0})
            Predict: 14887.56
          Else (feature 5 not in {3.0,4.0,5.0,6.0})
           If (feature 7 <= 56.05)
            Predict: 18692.0
           Else (feature 7 > 56.05)
            Predict: 18686.0
 ```

Your explanation here:

# Question 6 (5 pts)
Describe if the random forest model MSE score was better or worse than the MSE score from you best model in homework 3.  Include both scores in your description.

Your improvement explanation here:  

In HW3, the MSE score is 1542044.4275738671. \\
In RF model, the MSE is 509707.09139480535. \\
It becomes much better than the model in HW3. 






##### Grading Feedback Cell

# Question 7 (5 pts)
Set the `enable_grid_search` Boolean variable to False in the grading cell at the top of this notebook.  Perform a __Runtime -> factory reset__, __Runtime -> Run all__ test to verify there are no runtime errors.  Leave the `enable_grid_search` variable set to False and turn in your assignment.  This is the kind of thing you should be doing before you turn in every assignment. Remember this for future classes and when you get a job in industry.  This question will be graded as all or nothing.  You ether set the Boolean correct or not.  Additional points will be deducted elsewhere for runtime errors.

# Extra Credit (10 pts)
This homework was intended to take less time to complete and be about half the effort of previous assignments.  This doesn't allow us to explore GBT or deep learning.  

For extra credit, train a GBT or Deep Learning model using a grid search.  Protect the grid search inside the if enable_grid_search statement in the first code cell below.  You are free to use K-Fold cross validation if you wish.  The spark documentation for GBM can be found [here](https://spark.apache.org/docs/latest/ml-classification-regression.html#gradient-boosted-tree-classifier).  The spark documentation for deep learning can be found [here](https://spark.apache.org/docs/latest/ml-classification-regression.html#multilayer-perceptron-classifier)

In the second code cell below, hard code the best model parameters as determined by the grid search in a new pipeline named `best_pipe_2`.  Train and test `best_pipe_2` and save your resulting test MSE in a variable.  Do not use K-Fold cross validation when training best_pipe_2.  

In the third code cell below, create a pandas data frame named `compare_1_df` which contains 2 columns: Model and MSE.  Populate the Model column with model names: LR, RF, GBT or DL.  Populate the score column with the linear regression, random forest, and gradient boosted tree or deep learning test MSE scores. The linear regression score is from homework 3. The random forest score is from the random forest model above.  The GBT or Deep Learning score is from this extra credit problem.  Sort compare_1_df such that the best score is in the first row of the data frame. 

To get full credit, your GBT or deep learning solution should produce a score as good or better than the random forest score above.  In addition, the same rules as above apply where all of your grid search code shall be protected by the enable_grid_search Boolean.  Code that produces a runtime error when enable_grid_search is set to False will get 0 credit.

In [ ]:
# Your GBT / Deep Learning grid search code here
if enable_grid_search:
  pass

In [ ]:
# your hard coded parameter best model code here

In [ ]:
# Create compare_1_df

In [ ]:
# Grading cell do not modify
display(compare_1_df)